# Install package

In [ ]:
!conda create -n genet python=3.8
!conda activate genet

# install genet package in your env.
!pip3 install genet -f https://download.pytorch.org/whl/cu113/torch_stable.html git+https://github.com/Goosang-Yu/genet-models.git

# install ViennaRNA package for prediction module
#!pip install viennarna

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Cloning https://github.com/Goosang-Yu/genet-models.git to /tmp/pip-req-build-cq5vcocz
  Running command git clone --filter=blob:none --quiet https://github.com/Goosang-Yu/genet-models.git /tmp/pip-req-build-cq5vcocz
  Resolved https://github.com/Goosang-Yu/genet-models.git to commit b2cab377046542d665a17c3e6efcae971c9926f6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

 basic RNA secondary structure analysis software-                               مجموعه‌ای از ابزارها و کتابخانه‌های مستقل برای پیش‌بینی و تحلیل ساختار دوم اسید نوکلئیک است.

In [ ]:
!pip3 install ViennaRNA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 10.9 MB/s eta 0:00:00


In [ ]:
!pip3 install biopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip3 install RNA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
from genet import predict as prd

# Encoding

In [ ]:
#اینکدینگ مرحله 2----برای این 2 مرحله کردم که بتوانم تاثیر روش اینکدینگ خودم رو بسنجم
import numpy as np

class Encoder:
    def __init__(self, seq_wt, seq_et, with_category = False, label = None, with_reg_val = False, value = None):
        tlen = 24
        self.seq_wt = "-" *(tlen-len(seq_wt)) +  seq_wt
        self.seq_et = "-" *(tlen-len(seq_et)) + seq_et
        self.encoded_dict_indel = {'A': [1, 0, 0, 0, 0], 'T': [0, 1, 0, 0, 0],
                                   'G': [0, 0, 1, 0, 0], 'C': [0, 0, 0, 1, 0], '_': [0, 0, 0, 0, 1], '-': [0, 0, 0, 0, 0]}
        self.direction_dict = {'A':5, 'G':4, 'C':3, 'T':2, '_':1}
        if with_category:
            self.label = label
        if with_reg_val:
            self.value = value
        self.encode_wt_ed()

    def encode_seq_wt(self):
        code_list = []
        encoded_dict = self.encoded_dict_indel
        sgRNA_bases = list(self.seq_wt)
        for i in range(len(sgRNA_bases)):
            if sgRNA_bases[i] == "N":
                sgRNA_bases[i] = list(self.off_seq)[i]
            code_list.append(encoded_dict[sgRNA_bases[i]])
        self.sgRNA_code = np.array(code_list)

    def encode_seq_et(self):
        code_list = []
        encoded_dict = self.encoded_dict_indel
        off_bases = list(self.seq_et)
        for i in range(len(off_bases)):
            code_list.append(encoded_dict[off_bases[i]])
        self.off_code = np.array(code_list)

    def encode_wt_ed(self):
        self.encode_seq_wt()
        self.encode_seq_et()
        on_bases = list(self.seq_wt)
        off_bases = list(self.seq_et)
        on_off_dim7_codes = []
        for i in range(len(on_bases)):
            diff_code = np.bitwise_or(self.sgRNA_code[i], self.off_code[i])
            on_b = on_bases[i]
            off_b = off_bases[i]
            if on_b == "N":
                on_b = off_b

            dir_code = np.zeros(2)
            if on_b == "-" or off_b == "-" or self.direction_dict[on_b] == self.direction_dict[off_b]:
                pass
            else:
                if self.direction_dict[on_b] > self.direction_dict[off_b]:
                    dir_code[0] = 1
                else:
                    dir_code[1] = 1
            on_off_dim7_codes.append(np.concatenate((diff_code, dir_code)))
        self.on_off_code = np.array(on_off_dim7_codes)





چک اینکدینگ

In [ ]:
e = Encoder(seq_wt="AGCTGATTTTA", seq_et="CG_GTTTTTTG")
print(e.on_off_code)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 1. 0.]
 [1. 1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0.]]


# Embedding

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import DistilBertForSequenceClassification

DNABERTmodel = DistilBertForSequenceClassification.from_pretrained('Peltarion/dnabert-distilbert')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Peltarion/dnabert-distilbert and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Main function for Prime Editing

In [ ]:
#from genet.predict.models import DeepSpCas9, DeepPrime

In [ ]:
# from genet.utils import *
import genet
import genet.utils

import torch
import torch.nn.functional as F
import torch.nn as nn

import inspect



import os, sys, time, regex, logging
import numpy as np
import pandas as pd

import tensorflow.compat.v1 as tf

from glob import glob
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils import gc_fraction as gc
from Bio.Seq import Seq
from RNA import fold_compound  #compute minimum free energy (mfe)

np.set_printoptions(threshold=sys.maxsize)
tf.disable_v2_behavior()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'





#تعریف مدل در کریسپر

class Deep_xCas9(object):
    def __init__(self, filter_size, filter_num, node_1=80, node_2=60, l_rate=0.005):
        length = 30
        self.inputs = tf.placeholder(tf.float32, [None, 1, length, 4])
        self.targets = tf.placeholder(tf.float32, [None, 1])
        self.is_training = tf.placeholder(tf.bool)


#این لایه ها با توجه به مدل انتخاب شده برای اسکور دهی تعریف می شوند."

        def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
            # setup the filter input shape for tf.nn.conv_2d
            conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                               num_filters]

            # initialise weights and bias for the filter
            weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03), name=name + '_W')
            bias = tf.Variable(tf.truncated_normal([num_filters]), name=name + '_b')

            # setup the convolutional layer operation
            out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='VALID')

            # add the bias
            out_layer += bias

            # apply a ReLU non-linear activation
            out_layer = tf.layers.dropout(tf.nn.relu(out_layer), 0.3, self.is_training)

            # now perform max pooling
            ksize = [1, pool_shape[0], pool_shape[1], 1]
            strides = [1, 1, 2, 1]
            out_layer = tf.nn.avg_pool(out_layer, ksize=ksize, strides=strides, padding='SAME')

            return out_layer

        # def end: create_new_conv_layer

        L_pool_0 = create_new_conv_layer(self.inputs, 4, filter_num[0], [1, filter_size[0]], [1, 2], name='conv1')
        L_pool_1 = create_new_conv_layer(self.inputs, 4, filter_num[1], [1, filter_size[1]], [1, 2], name='conv2')
        L_pool_2 = create_new_conv_layer(self.inputs, 4, filter_num[2], [1, filter_size[2]], [1, 2], name='conv3')

        with tf.variable_scope('Fully_Connected_Layer1'):
            layer_node_0 = int((length - filter_size[0]) / 2) + 1
            node_num_0   = layer_node_0 * filter_num[0]
            layer_node_1 = int((length - filter_size[1]) / 2) + 1
            node_num_1   = layer_node_1 * filter_num[1]
            layer_node_2 = int((length - filter_size[2]) / 2) + 1
            node_num_2   = layer_node_2 * filter_num[2]

            L_flatten_0  = tf.reshape(L_pool_0, [-1, node_num_0])
            L_flatten_1  = tf.reshape(L_pool_1, [-1, node_num_1])
            L_flatten_2  = tf.reshape(L_pool_2, [-1, node_num_2])
            L_flatten    = tf.concat([L_flatten_0, L_flatten_1, L_flatten_2], 1, name='concat')

            node_num     = node_num_0 + node_num_1 + node_num_2
            W_fcl1       = tf.get_variable("W_fcl1", shape=[node_num, node_1])
            B_fcl1       = tf.get_variable("B_fcl1", shape=[node_1])
            L_fcl1_pre   = tf.nn.bias_add(tf.matmul(L_flatten, W_fcl1), B_fcl1)
            L_fcl1       = tf.nn.relu(L_fcl1_pre)
            L_fcl1_drop  = tf.layers.dropout(L_fcl1, 0.3, self.is_training)

        with tf.variable_scope('Fully_Connected_Layer2'):
            W_fcl2       = tf.get_variable("W_fcl2", shape=[node_1, node_2])
            B_fcl2       = tf.get_variable("B_fcl2", shape=[node_2])
            L_fcl2_pre   = tf.nn.bias_add(tf.matmul(L_fcl1_drop, W_fcl2), B_fcl2)
            L_fcl2       = tf.nn.relu(L_fcl2_pre)
            L_fcl2_drop  = tf.layers.dropout(L_fcl2, 0.3, self.is_training)

        with tf.variable_scope('Output_Layer'):
            W_out        = tf.get_variable("W_out", shape=[node_2, 1])
            B_out        = tf.get_variable("B_out", shape=[1])
            self.outputs = tf.nn.bias_add(tf.matmul(L_fcl2_drop, W_out), B_out)

        # Define loss function and optimizer
        self.obj_loss    = tf.reduce_mean(tf.square(self.targets - self.outputs))
        self.optimizer   = tf.train.AdamOptimizer(l_rate).minimize(self.obj_loss)

    # def end: def __init__
# class end: Deep_xCas9

#این تابع با توجه به مدل از پیش آموزش دیده اسکور همه رشته های طراحی شده را حساب می کند."
def Model_Finaltest(sess, TEST_X, model):
    test_batch = 500
    TEST_Z = np.zeros((TEST_X.shape[0], 1), dtype=float)

# به ازای هر 500 تا از مدل تعریف شده در بالا درخواست خروجی می کنیم.
    for i in range(int(np.ceil(float(TEST_X.shape[0]) / float(test_batch)))):
        Dict = {model.inputs: TEST_X[i * test_batch:(i + 1) * test_batch], model.is_training: False}
        TEST_Z[i * test_batch:(i + 1) * test_batch] = sess.run([model.outputs], feed_dict=Dict)[0]
#  این برای حالتی است که طول دنباله بیش از 500 است ولی طول دنباله من 63 است و این جمع در واقع جمع اسکور عوامل تاثیرگذار است.
    list_score = sum(TEST_Z.tolist(), [])

    return list_score


# def end: Model_Finaltest


#این تابع برای  اینکدینگ مرحله 1 است"
def preprocess_seq(data, seq_length):

    seq_onehot = np.zeros((len(data), 1, seq_length, 4), dtype=float)

    for l in range(len(data)):
        for i in range(seq_length):
            try:
                data[l][i]
            except Exception:
                print(data[l], i, seq_length, len(data))

            if   data[l][i] in "Aa":  seq_onehot[l, 0, i, 0] = 1
            elif data[l][i] in "Cc":  seq_onehot[l, 0, i, 1] = 1
            elif data[l][i] in "Gg":  seq_onehot[l, 0, i, 2] = 1
            elif data[l][i] in "Tt":  seq_onehot[l, 0, i, 3] = 1
            elif data[l][i] in "Xx":  pass
            elif data[l][i] in "Nn.": pass
            else:
                print("[Input Error] Non-ATGC character " + data[l])
                sys.exit()

    return seq_onehot

#  برای فراخوانی تابعی که در بالال برای اینکدینگ تعریف کردم
def preprocess_seq2(seq_wt, seq_et):

    e = Encoder(seq_wt, seq_et)

    #self.on_off_code = np.array(on_off_dim7_codes)
    #return self.on_off_code


#"اسکور دهی بر اساس کریسپر و مدل انتخابی"
def spcas9_score(list_target30:list , gpu_env=0):
    '''
    input:: list_target  with length 30 n
    The list_target30 should have a 30bp sequence in the form of a list.
    Also, sequence [24:27] should contain NGG PAM.
    >>> list_out = spcas9_score(list_target30)


'''
  #  best_model را تغییر دادم در ادامه شاید "
  #در حال حاضر از مدل سایر محققین استفاده می کنم"

  # TensorFlow config
    conf = tf.ConfigProto()
    conf.gpu_options.allow_growth = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '%d' % gpu_env

    x_test = preprocess_seq(list_target30, 30)

    from genet_models import load_



    model_dir = load_deepspcas9()

    best_model_path = model_dir
    best_model = 'PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60'

    model_save = '%s/%s' % (best_model_path, best_model)
# شبکه کانولوشنی که در بالا تعریف کردم براساس این پارامترها ساخته می شود.
    filter_size = [3, 5, 7]
    filter_num  = [100, 70, 40]
    args        = [filter_size, filter_num, 0.001, 550]

    tf.reset_default_graph()

    with tf.Session(config=conf) as sess:
        sess.run(tf.global_variables_initializer())
        model = Deep_xCas9(filter_size, filter_num, 80, 60, args[2])

        saver = tf.train.Saver()
        saver.restore(sess, model_save)
# تابع اسکور دهی در اینجا و بعد مشخص کردم مدل انتخابی فراخوانی می شود.
        list_score = Model_Finaltest(sess, x_test, model)

    return list_score











#تعریف مدل در پرایم ادیتینگ

#" 3اتابع زیر برای کارهای پرایم ادیتینگ است ولی نمی دانم دقیقا چه می کنند ولی بودنشان برای ساخت تمام رشته های راهنمایی ممکن  ضروری است."
# پیش پردازش برای پرایم ادیتینگ
def reverse_complement(sSeq):
    dict_sBases = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N', 'U': 'U', 'n': '',
                   '.': '.', '*': '*', 'a': 't', 'c': 'g', 'g': 'c', 't': 'a'}
    list_sSeq = list(sSeq)  # Turns the sequence in to a gigantic list
    list_sSeq = [dict_sBases[sBase] for sBase in list_sSeq]
    return ''.join(list_sSeq)[::-1]

# def END: reverse_complement

def set_alt_position_window(sStrand, sAltKey, nAltIndex, nIndexStart, nIndexEnd, nAltLen):
    if sStrand == '+':

        if sAltKey.startswith('sub'):
            return (nAltIndex + 1) - (nIndexStart - 3)
        else:
            return (nAltIndex + 1) - (nIndexStart - 3)

    else:
        if sAltKey.startswith('sub'):
            return nIndexEnd - nAltIndex + 3 - (nAltLen - 1)

        elif sAltKey.startswith('del'):
            return nIndexEnd - nAltIndex + 3 - nAltLen

        else:
            return nIndexEnd - nAltIndex + 3 + nAltLen
        # if END:
    # if END:

# def END: set_alt_position_window


def set_PAM_nicking_pos(sStrand, sAltType, nAltLen, nAltIndex, nIndexStart, nIndexEnd):
    if sStrand == '-':
        nPAM_Nick = nIndexEnd + 3
    else:
        nPAM_Nick = nIndexStart - 3

    return nPAM_Nick

# def END: set_PAM_Nicking_Pos


def check_PAM_window(dict_sWinSize, sStrand, nIndexStart, nIndexEnd, sAltType, nAltLen, nAltIndex):
    nUp, nDown = dict_sWinSize[sAltType][nAltLen]

    if sStrand == '+':
        nPAMCheck_min = nAltIndex - nUp + 1
        nPAMCheck_max = nAltIndex + nDown + 1
    else:
        nPAMCheck_min = nAltIndex - nDown + 1
        nPAMCheck_max = nAltIndex + nUp + 1
    # if END:

    if nIndexStart < nPAMCheck_min or nIndexEnd > nPAMCheck_max:
        return 0
    else:
        return 1

# def END: check_PAM_window



#ویژگی های دخیل در دقت ویرایش پرایم ادیتینگ استخراج و  بر اساس آنها اسکور نهایی پرایم ادیتینگ حساب می شود."
class FeatureExtraction:
    def __init__(self):
        self.sGuideKey = ''
        self.sChrID = ''
        self.sStrand = ''
        self.nGenomicPos = 0
        self.nEditIndex = 0
        self.nPBSLen = 0
        self.nRTTLen = 0
        self.sPBSSeq = ''
        self.sRTSeq = ''
        self.sPegRNASeq = ''
        self.sWTSeq = ''
        self.sEditedSeq = ''
        self.list_sSeqs = []
        self.type_sub = 0
        self.type_ins = 0
        self.type_del = 0
        self.fTm1 = 0.0 # temp of melting
        self.fTm2 = 0.0
        self.fTm2new = 0.0
        self.fTm3 = 0.0
        self.fTm4 = 0.0
        self.fTmD = 0.0
        self.fMFE3 = 0.0 # minimum free energy (mfe)
        self.fMFE4 = 0.0
        self.nGCcnt1 = 0
        self.nGCcnt2 = 0
        self.nGCcnt3 = 0
        self.fGCcont1 = 0.0
        self.fGCcont2 = 0.0
        self.fGCcont3 = 0.0
        self.dict_sSeqs = {}
        self.dict_sCombos = {}
        self.dict_sOutput = {}

    # def End: __init__



  #"   با توجه به ورودی که از کاربر می گیریم متغییر ها مقدار دهی می شود  من فعلا 3 متغییر را می گیرم "
  #"بعد تعریف پوسته برنامه این قسمت تغییر خواهد کرد."
    def get_input(self, wt_seq, ed_seq, edit_type, edit_len):
        self.sWTSeq = wt_seq.upper()
        self.sEditedSeq = ed_seq.upper()
        self.sAltKey = edit_type + str(edit_len)
        self.sAltType = edit_type
        self.nAltLen = edit_len

        if   self.sAltType.startswith('sub'): self.type_sub = 1
        elif self.sAltType.startswith('del'): self.type_del = 1
        elif self.sAltType.startswith('ins'): self.type_ins = 1

    # def End: get_input




   #"بعد گرفتن پارامترها از کاربر تمام رشته های راهنمای ممکن باید طراحی و سپس اسکور دهی شوند."
   #"برای ساخت تمام رشته های ممکن از کد های گیتاپ استفاده کرده ام."

   #" RT_PBS اول تمام "
   #"pegRNA بعد تمام "

    def get_sAltNotation(self, nAltIndex):
        if self.sAltType == 'sub':
            self.sAltNotation = '%s>%s' % (
                self.sWTSeq[nAltIndex:nAltIndex + self.nAltLen], self.sEditedSeq[nAltIndex:nAltIndex + self.nAltLen])

        elif self.sAltType == 'del':
            self.sAltNotation = '%s>%s' % (
                self.sWTSeq[nAltIndex:nAltIndex + 1 + self.nAltLen], self.sEditedSeq[nAltIndex])

        else:
            self.sAltNotation = '%s>%s' % (
                self.sWTSeq[nAltIndex], self.sEditedSeq[nAltIndex:nAltIndex + self.nAltLen + 1])

    # def END: get_sAltNotation

    def get_all_RT_PBS(self,
                    nAltIndex,
                    nMinPBS = 0,
                    nMaxPBS = 17,
                    nMaxRT = 40,
                    nSetPBSLen = 0,
                    nSetRTLen = 0,
                    pe_system = 'PE2'
                    ):
        """
        nMinPBS: If you set specific number, lower than MinPBS will be not generated. Default=0
        nMaxPBS: If you set specific number, higher than MinPBS will be not generated. Default=17
        nMaxRT = : If you set specific number, higher than MinPBS will be not generated. Default=40
        nSetPBSLen = 0  # Fix PBS Len: Set if >0
        nSetRTLen = 0  # Fix RT  Len: Set if >0
        PAM: 4-nt sequence
        """

        nMaxEditPosWin = nMaxRT + 3  # Distance between PAM and mutation

        dict_sWinSize = {'sub': {1: [nMaxRT - 1 - 3, 6], 2: [nMaxRT - 2 - 3, 6], 3: [nMaxRT - 3 - 3, 6]},
                        'ins': {1: [nMaxRT - 2 - 3, 6], 2: [nMaxRT - 3 - 3, 6], 3: [nMaxRT - 4 - 3, 6]},
                        'del': {1: [nMaxRT - 1 - 3, 6], 2: [nMaxRT - 1 - 3, 6], 3: [nMaxRT - 1 - 3, 6]}}


        if 'NRCH' in pe_system: # for NRCH-PE PAM
            dict_sRE = {'+': '[ACGT][ACGT]G[ACGT]|[ACGT][CG]A[ACGT]|[ACGT][AG]CC|[ATCG]ATG',
                        '-': '[ACGT]C[ACGT][ACGT]|[ACGT]T[CG][ACGT]|G[GT]T[ACGT]|ATT[ACGT]|CAT[ACGT]|GGC[ACGT]|GTA[ACGT]'}
        else:
            dict_sRE = {'+': '[ACGT]GG[ACGT]', '-': '[ACGT]CC[ACGT]'} # for Original-PE PAM

        for sStrand in ['+', '-']:

            sRE = dict_sRE[sStrand]
            for sReIndex in regex.finditer(sRE, self.sWTSeq, overlapped=True):

                if sStrand == '+':
                    nIndexStart = sReIndex.start()
                    nIndexEnd = sReIndex.end() - 1
                    sPAMSeq = self.sWTSeq[nIndexStart:nIndexEnd]
                    sGuideSeq = self.sWTSeq[nIndexStart - 20:nIndexEnd]
                else:
                    nIndexStart = sReIndex.start() + 1
                    nIndexEnd = sReIndex.end()
                    sPAMSeq = reverse_complement(self.sWTSeq[nIndexStart:nIndexEnd])
                    sGuideSeq = reverse_complement(self.sWTSeq[nIndexStart:nIndexEnd + 20])

                nAltPosWin = set_alt_position_window(sStrand, self.sAltKey, nAltIndex, nIndexStart, nIndexEnd,
                                                    self.nAltLen)

                ## AltPosWin Filter ##
                if nAltPosWin <= 0:             continue
                if nAltPosWin > nMaxEditPosWin: continue

                nPAM_Nick = set_PAM_nicking_pos(sStrand, self.sAltType, self.nAltLen, nAltIndex, nIndexStart, nIndexEnd)

                if not check_PAM_window(dict_sWinSize, sStrand, nIndexStart, nIndexEnd, self.sAltType, self.nAltLen,
                                        nAltIndex): continue

                sPAMKey = '%s,%s,%s,%s,%s,%s,%s' % (
                    self.sAltKey, self.sAltNotation, sStrand, nPAM_Nick, nAltPosWin, sPAMSeq, sGuideSeq)

                dict_sRT, dict_sPBS = self.determine_PBS_RT_seq(sStrand, nMinPBS, nMaxPBS, nMaxRT, nSetPBSLen,
                                                        nSetRTLen, nAltIndex, nPAM_Nick, nAltPosWin, self.sEditedSeq)

                nCnt1, nCnt2 = len(dict_sRT), len(dict_sPBS)
                if nCnt1 == 0: continue
                if nCnt2 == 0: continue

                if sPAMKey not in self.dict_sSeqs:
                    self.dict_sSeqs[sPAMKey] = ''
                self.dict_sSeqs[sPAMKey] = [dict_sRT, dict_sPBS]

            # loop END: sReIndex
        # loop END: sStrand


    # def END: get_all_RT_PBS



 #"pegRNA تمام "

    def determine_PBS_RT_seq(self, sStrand, nMinPBS, nMaxPBS, nMaxRT, nSetPBSLen, nSetRTLen, nAltIndex, nPAM_Nick,
                            nAltPosWin, sForTempSeq):
        dict_sPBS = {}
        dict_sRT = {}

        list_nPBSLen = [nNo + 1 for nNo in range(nMinPBS, nMaxPBS)]
        for nPBSLen in list_nPBSLen:

            ## Set PBS Length ##
            if nSetPBSLen:
                if nPBSLen != nSetPBSLen: continue

            if sStrand == '+':
                nPBSStart = nPAM_Nick - nPBSLen  # 5' -> PamNick
                nPBSEnd = nPAM_Nick
                sPBSSeq = sForTempSeq[nPBSStart:nPBSEnd] # sForTempSeq = self.EditedSeq

            else:
                if self.sAltKey.startswith('sub'):
                    nPBSStart = nPAM_Nick
                elif self.sAltKey.startswith('ins'):
                    nPBSStart = nPAM_Nick + self.nAltLen
                elif self.sAltKey.startswith('del'):
                    nPBSStart = nPAM_Nick - self.nAltLen

                sPBSSeq = reverse_complement(sForTempSeq[nPBSStart:nPBSStart + nPBSLen]) # sForTempSeq = self.EditedSeq

            # if END: sStrand

            sKey = len(sPBSSeq)
            if sKey not in dict_sPBS:
                dict_sPBS[sKey] = ''
            dict_sPBS[sKey] = sPBSSeq
        # loop END: nPBSLen

        if sStrand == '+':
            if self.sAltKey.startswith('sub'):
                list_nRTPos = [nNo + 1 for nNo in range(nAltIndex + self.nAltLen, (nPAM_Nick + nMaxRT))]
            elif self.sAltKey.startswith('ins'):
                list_nRTPos = [nNo + 1 for nNo in range(nAltIndex + self.nAltLen, (nPAM_Nick + nMaxRT))]
            else:
                list_nRTPos = [nNo + 1 for nNo in range(nAltIndex, (nPAM_Nick + nMaxRT))]
        else:
            if self.sAltKey.startswith('sub'):
                list_nRTPos = [nNo for nNo in range(nPAM_Nick - 1 - nMaxRT, nAltIndex)]
            else:
                list_nRTPos = [nNo for nNo in range(nPAM_Nick - 3 - nMaxRT, nAltIndex + self.nAltLen - 1)]
        for nRTPos in list_nRTPos:

            if sStrand == '+':
                nRTStart = nPAM_Nick  # PamNick -> 3'
                nRTEnd = nRTPos
                sRTSeq = sForTempSeq[nRTStart:nRTEnd]

            else:
                if self.sAltKey.startswith('sub'):
                    nRTStart = nRTPos
                    nRTEnd = nPAM_Nick  # PamNick -> 3'
                elif self.sAltKey.startswith('ins'):
                    nRTStart = nRTPos
                    nRTEnd = nPAM_Nick + self.nAltLen  # PamNick -> 3'
                elif self.sAltKey.startswith('del'):
                    nRTStart = nRTPos
                    nRTEnd = nPAM_Nick - self.nAltLen  # PamNick -> 3'

                sRTSeq = reverse_complement(sForTempSeq[nRTStart:nRTEnd])

                if not sRTSeq: continue
            # if END: sStrand

            sKey = len(sRTSeq)

            ## Set RT Length ##
            if nSetRTLen:
                if sKey != nSetRTLen: continue

            ## Limit Max RT len ##
            if sKey > nMaxRT: continue

            ## min RT from nick site to mutation ##
            if self.sAltKey.startswith('sub'):
                if sStrand == '+':
                    if sKey < abs(nAltIndex - nPAM_Nick): continue
                else:
                    if sKey < abs(nAltIndex - nPAM_Nick + self.nAltLen - 1): continue ###
            else:
                if sStrand == '-':
                    if sKey < abs(nAltIndex - nPAM_Nick + self.nAltLen - 1): continue

            if self.sAltKey.startswith('ins'):
                if sKey < nAltPosWin + 1: continue

            if sKey not in dict_sRT:
                dict_sRT[sKey] = ''
            dict_sRT[sKey] = sRTSeq
        # loop END: nRTPos

        return [dict_sRT, dict_sPBS]


    # def END: determine_PBS_RT_seq







    def make_rt_pbs_combinations(self):
        for sPAMKey in self.dict_sSeqs:

            dict_sRT, dict_sPBS = self.dict_sSeqs[sPAMKey]

            list_sRT = [dict_sRT[sKey] for sKey in dict_sRT]
            list_sPBS = [dict_sPBS[sKey] for sKey in dict_sPBS]

            if sPAMKey not in self.dict_sCombos:
                self.dict_sCombos[sPAMKey] = ''
            self.dict_sCombos[sPAMKey] = {'%s,%s' % (sRT, sPBS): {} for sRT in list_sRT for sPBS in list_sPBS}
        # loop END: sPAMKey


    # def END: make_rt_pbs_combinations

# استخراج ویژگی های مهم دنباله های ورودی برای ساختار دوم
    def determine_seqs(self):
        for sPAMKey in self.dict_sSeqs:

            sAltKey, sAltNotation, sStrand, nPAM_Nick, nAltPosWin, sPAMSeq, sGuideSeq = sPAMKey.split(',')
            nAltPosWin = int(nAltPosWin)
            nNickIndex = int(nPAM_Nick)

            # if sStrand == '+':
            #     sWTSeq74 = self.sWTSeq[nNickIndex - 21:nNickIndex + 53]
            # else:
            #     sWTSeq74 = reverse_complement(self.sWTSeq[nNickIndex - 53:nNickIndex + 21])

            for sSeqKey in self.dict_sCombos[sPAMKey]:

                sRTSeq, sPBSSeq = sSeqKey.split(',')

                ## for Tm1
                sForTm1 = reverse_complement(sPBSSeq.replace('A', 'U'))

                if sStrand == '+':
                    ## for Tm2
                    sForTm2 = self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq)]

                    ## for Tm2new
                    if self.sAltType.startswith('sub'):
                        sForTm2new = self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq)]
                    elif self.sAltType.startswith('ins'):
                        sForTm2new = self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq) - self.nAltLen]
                    else:  # del
                        sForTm2new = self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq) + self.nAltLen]

                    ## for Tm3
                    if self.sAltType.startswith('sub'):
                        sTm3antiSeq = reverse_complement(self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq)])
                    elif self.sAltType.startswith('ins'):
                        sTm3antiSeq = reverse_complement(self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq) - self.nAltLen])
                    else:  # del
                        sTm3antiSeq = reverse_complement(self.sWTSeq[nNickIndex:nNickIndex + len(sRTSeq) + self.nAltLen])

                else:
                    ## for Tm2
                    sForTm2 = reverse_complement(self.sWTSeq[nNickIndex - len(sRTSeq):nNickIndex])

                    ## for Tm2new
                    if self.sAltType.startswith('sub'):
                        sForTm2new = reverse_complement(self.sWTSeq[nNickIndex - len(sRTSeq):nNickIndex])
                    elif self.sAltType.startswith('ins'):
                        sForTm2new = reverse_complement(self.sWTSeq[nNickIndex - len(sRTSeq) + self.nAltLen:nNickIndex])
                    else:  # del
                        sForTm2new = reverse_complement(self.sWTSeq[nNickIndex - len(sRTSeq) - self.nAltLen:nNickIndex])

                    ## for Tm3
                    if self.sAltType.startswith('sub'):
                        sTm3antiSeq = self.sWTSeq[nNickIndex - len(sRTSeq):nNickIndex]
                    elif self.sAltType.startswith('ins'):
                        sTm3antiSeq = self.sWTSeq[nNickIndex - len(sRTSeq) + self.nAltLen:nNickIndex]
                    else:  # del
                        sTm3antiSeq = self.sWTSeq[nNickIndex - len(sRTSeq) - self.nAltLen:nNickIndex]

                # if END

                sForTm3 = [sRTSeq, sTm3antiSeq]

                ## for Tm4
                sForTm4 = [reverse_complement(sRTSeq.replace('A', 'U')), sRTSeq]


                self.dict_sCombos[sPAMKey][sSeqKey] = {'Tm1': sForTm1,
                                                        'Tm2': sForTm2,
                                                        'Tm2new': sForTm2new,
                                                        'Tm3': sForTm3,
                                                        'Tm4': sForTm4}
            # loop END: sSeqKey
        # loop END: sPAMKey
    # def END: determine_seqs


#" تا اینجا تمام رشته های  PegRNA را تعریف کردم"


#  "حال ساختار دوم رشته ها "
#   "برای اسکور دهی کلا 3 نوع پارامتر در نظر گرفته می شود ویژگی های رشته های ورودی ، ساختار دوم و اینتراکشن ژن ها"
#"تا اینجا داشتیم  پارامترهای تاثیر گذار  را از ساختار رشته های ورودی استخراج می کردیم
#" در ادامه ویژگی های مربوط به ساختار دوم ژن بررسی می شود."

    def determine_secondary_structure(self):
        for sPAMKey in self.dict_sSeqs:

            sAltKey, sAltNotation, sStrand, nPAM_Nick, nAltPosWin, sPAMSeq, sGuideSeq = sPAMKey.split(',')
            list_sOutputKeys = ['Tm1', 'Tm2', 'Tm2new', 'Tm3', 'Tm4', 'TmD', 'nGCcnt1', 'nGCcnt2', 'nGCcnt3',
                        'fGCcont1', 'fGCcont2', 'fGCcont3', 'MFE3', 'MFE4']

            if sPAMKey not in self.dict_sOutput:
                self.dict_sOutput[sPAMKey] = {}

            for sSeqKey in self.dict_sCombos[sPAMKey]:

                if sSeqKey not in self.dict_sOutput[sPAMKey]:

                    self.dict_sOutput[sPAMKey][sSeqKey] = {sKey: '' for sKey in list_sOutputKeys}

                self.determine_Tm(sPAMKey, sSeqKey)
                self.determine_GC(sPAMKey, sSeqKey)
                self.determine_MFE(sPAMKey, sSeqKey, sGuideSeq)
            # loop END: sSeqKey
        # loop END: sPAMKey



#"بررسی اینتراکشن ها "
    def determine_Tm(self, sPAMKey, sSeqKey):
        sForTm1 = self.dict_sCombos[sPAMKey][sSeqKey]['Tm1']
        sForTm2 = self.dict_sCombos[sPAMKey][sSeqKey]['Tm2']
        sForTm2new = self.dict_sCombos[sPAMKey][sSeqKey]['Tm2new']
        sForTm3 = self.dict_sCombos[sPAMKey][sSeqKey]['Tm3']
        sForTm4 = self.dict_sCombos[sPAMKey][sSeqKey]['Tm4']

        ## Tm1 DNA/RNA mm1 ##
        fTm1 = mt.Tm_NN(seq=Seq(sForTm1), nn_table=mt.R_DNA_NN1)

        ## Tm2 DNA/DNA mm0 ##
        fTm2 = mt.Tm_NN(seq=Seq(sForTm2), nn_table=mt.DNA_NN3)

        ## Tm2new DNA/DNA mm0 ##
        fTm2new = mt.Tm_NN(seq=Seq(sForTm2new), nn_table=mt.DNA_NN3)

        ## Tm3 DNA/DNA mm1 ##
        if not sForTm3:
            fTm3 = 0
            fTm5 = 0

        else:
            list_fTm3 = []
            for sSeq1, sSeq2 in zip(sForTm3[0], sForTm3[1]):
                try:
                    fTm3 = mt.Tm_NN(seq=sSeq1, c_seq=sSeq2, nn_table=mt.DNA_NN3)
                except ValueError:
                    continue

                list_fTm3.append(fTm3)
            # loop END: sSeq1, sSeq2

        # if END:

        # Tm4 - revcom(AAGTcGATCC(RNA version)) + AAGTcGATCC
        fTm4 = mt.Tm_NN(seq=Seq(sForTm4[0]), nn_table=mt.R_DNA_NN1)

        # Tm5 - Tm3 - Tm2
        fTm5 = fTm3 - fTm2

        self.dict_sOutput[sPAMKey][sSeqKey]['Tm1'] = fTm1
        self.dict_sOutput[sPAMKey][sSeqKey]['Tm2'] = fTm2
        self.dict_sOutput[sPAMKey][sSeqKey]['Tm2new'] = fTm2new
        self.dict_sOutput[sPAMKey][sSeqKey]['Tm3'] = fTm3
        self.dict_sOutput[sPAMKey][sSeqKey]['Tm4'] = fTm4
        self.dict_sOutput[sPAMKey][sSeqKey]['TmD'] = fTm5

    # def END: determine_Tm


    def determine_GC(self, sPAMKey, sSeqKey):
        sRTSeqAlt, sPBSSeq = sSeqKey.split(',')

        self.nGCcnt1 = sPBSSeq.count('G') + sPBSSeq.count('C')
        self.nGCcnt2 = sRTSeqAlt.count('G') + sRTSeqAlt.count('C')
        self.nGCcnt3 = (sPBSSeq + sRTSeqAlt).count('G') + (sPBSSeq + sRTSeqAlt).count('C')
        self.fGCcont1 = 100 * gc(sPBSSeq)
        self.fGCcont2 = 100 * gc(sRTSeqAlt)
        self.fGCcont3 = 100 * gc(sPBSSeq + sRTSeqAlt)
        self.dict_sOutput[sPAMKey][sSeqKey]['nGCcnt1'] = self.nGCcnt1
        self.dict_sOutput[sPAMKey][sSeqKey]['nGCcnt2'] = self.nGCcnt2
        self.dict_sOutput[sPAMKey][sSeqKey]['nGCcnt3'] = self.nGCcnt3
        self.dict_sOutput[sPAMKey][sSeqKey]['fGCcont1'] = self.fGCcont1
        self.dict_sOutput[sPAMKey][sSeqKey]['fGCcont2'] = self.fGCcont2
        self.dict_sOutput[sPAMKey][sSeqKey]['fGCcont3'] = self.fGCcont3


    # def END: determine_GC

    def determine_MFE(self, sPAMKey, sSeqKey, sGuideSeqExt):

        sRTSeq, sPBSSeq = sSeqKey.split(',')

        ## Set GuideRNA seq ##
        sGuideSeq = 'G' + sGuideSeqExt[1:-3] ## GN19 guide seq

        # MFE_3 - RT + PBS + PolyT
        sInputSeq = reverse_complement(sPBSSeq + sRTSeq) + 'TTTTTT'
        sDBSeq, fMFE3 = fold_compound(sInputSeq).mfe()   #mfe= prediction of minimume free energe which refers to  the termodinamic stability of pegRNA-DNA interactions based on the sequence and structual features
        # MFE_4 - spacer only
        sInputSeq = sGuideSeq
        sDBSeq, fMFE4 = fold_compound(sInputSeq).mfe()

        self.dict_sOutput[sPAMKey][sSeqKey]['MFE3'] = round(fMFE3, 1)
        self.dict_sOutput[sPAMKey][sSeqKey]['MFE4'] = round(fMFE4, 1)

    # def END: determine_MFE









#"این تابع برای مشخص  کردن ستون هایی است که در خروجی نمایش داده می شود."
#" هر یک از این پارامترها را در بالا تعریف و حساب کرده ایم و حال نمایش می دهیم."

#"برای همه  رشته ها که اسکور آنها را حساب کردم این خروجی ها حساب می شود ولی من فقط 10 تا را نمایش می دهم."


    def make_output_df(self):

        list_output = []
        list_sOutputKeys = ['Tm1', 'Tm2', 'Tm2new', 'Tm3', 'Tm4', 'TmD', 'nGCcnt1', 'nGCcnt2', 'nGCcnt3',
                        'fGCcont1', 'fGCcont2', 'fGCcont3', 'MFE3', 'MFE4']

        for sPAMKey in self.dict_sSeqs:

            sAltKey, sAltNotation, sStrand, nPAM_Nick, nAltPosWin, sPAMSeq, sGuideSeq = sPAMKey.split(',')
            nNickIndex = int(nPAM_Nick)

            if sStrand == '+':
                sWTSeq74 = self.sWTSeq[nNickIndex - 21:nNickIndex + 53]
                nEditPos = 61 - nNickIndex
            else:
                sWTSeq74 = reverse_complement(self.sWTSeq[nNickIndex - 53:nNickIndex + 21])
                if not self.sAltType.startswith('ins'):
                    nEditPos = nNickIndex - 60 - self.nAltLen + 1
                else:
                    nEditPos = nNickIndex - 59

            for sSeqKey in self.dict_sOutput[sPAMKey]:

                dict_seq = self.dict_sCombos[sPAMKey][sSeqKey]
                sRTTSeq, sPBSSeq = sSeqKey.split(',')
                PBSlen = len(sPBSSeq)
                RTlen = len(sRTTSeq)

                sPBS_RTSeq = sPBSSeq + sRTTSeq
                s5Bufferlen = 21 - PBSlen
                s3Bufferlen = 53 - RTlen
                sEDSeq74 = 'x' * s5Bufferlen + sPBS_RTSeq + 'x' * s3Bufferlen

                if self.sAltType.startswith('del'):
                    RHA_len = len(sRTTSeq) - nEditPos + 1
                else:
                    RHA_len = len(sRTTSeq) - nEditPos - self.nAltLen + 1


                list_sOut = [self.input_id, sWTSeq74, sEDSeq74,
                            len(sPBSSeq), len(sRTTSeq), len(sPBSSeq + sRTTSeq), nEditPos, self.nAltLen,
                            RHA_len, self.type_sub, self.type_ins, self.type_del
                            ] + [self.dict_sOutput[sPAMKey][sSeqKey][sKey] for sKey in list_sOutputKeys]

                list_output.append(list_sOut)

            # loop END: sSeqKey

        hder_essen = ['ID', 'WT74_On', 'Edited74_On', 'PBSlen', 'RTlen', 'RT-PBSlen', 'Edit_pos', 'Edit_len', 'RHA_len',
                    'type_sub', 'type_ins', 'type_del','Tm1', 'Tm2', 'Tm2new', 'Tm3', 'Tm4', 'TmD',
                    'nGCcnt1', 'nGCcnt2', 'nGCcnt3', 'fGCcont1', 'fGCcont2', 'fGCcont3', 'MFE3', 'MFE4']

        df_out = pd.DataFrame(list_output, columns=hder_essen)

        # loop END: sPAMKey

        return df_out

# def END: make_output

#"برای اسکور دهی کلا 3 نوع پارامتر در نظر گرفته می شود ویژگی های رشته های ورودی ، ساختار دوم و اینتراکشن ژن ها"
#"تا اینجا داشتیم  پارامترهای تاثیر گذار  را از ساختار رشته های ورودی استخراج و ساختار دوم می کردیم
#" در ادامه ویژگی های مربوط به اینتراکشن ژن ها بررسی می شود."






# ساخت یک شبکه CNN , GRU
#جدید برای پرایم

class GeneInteractionModel(nn.Module):


    def __init__(self, hidden_size, num_layers, num_features=24, dropout=0.1):
        super(GeneInteractionModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.c1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=128, kernel_size=(2, 3), stride=1, padding=(0, 1)),
            nn.BatchNorm2d(128),
            nn.GELU(),
        )
        self.c2 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=108, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(108),
            nn.GELU(),
            nn.AvgPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=108, out_channels=108, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(108),
            nn.GELU(),
            nn.AvgPool1d(kernel_size=2, stride=2),

            nn.Conv1d(in_channels=108, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.AvgPool1d(kernel_size=2, stride=2),
        )

        self.r = nn.GRU(128, hidden_size, num_layers, batch_first=True, bidirectional=True)

        self.s = nn.Linear(2 * hidden_size, 12, bias=False)

        self.d = nn.Sequential(
            nn.Linear(num_features, 96, bias=False),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(96, 64, bias=False),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 128, bias=False)
        )

        self.head = nn.Sequential(
            nn.BatchNorm1d(140),
            nn.Dropout(dropout),
            nn.Linear(140, 1, bias=True),
        )

    def forward(self, g, x):
        g = torch.squeeze(self.c1(g), 2)
        g = self.c2(g)
        g, _ = self.r(torch.transpose(g, 1, 2))
        g = self.s(g[:, -1, :])

        x = self.d(x)

        out = self.head(torch.cat((g, x), dim=1))

        return F.softplus(out)




# شروع پایپ لاین اصلی
#ورودی هایی که از کاربر گرفتیم را به وان هات تبدیل می کند
def seq_concat(data, col1='WT74_On', col2='Edited74_On', seq_length=74):
#   wt = preprocess_seq2(data[col1], seq_length)
#   ed = preprocess_seq2(data[col2], seq_length)

    wt = preprocess_seq(data[col1], seq_length)
    ed = preprocess_seq(data[col2], seq_length)
    g = np.concatenate((wt, ed), axis=1)
    g = 2 * g - 1

    return g

 #       1-the Tm (melting temperature) of the DNA:RNA hybrid from positions 16 - 20 of the sgRNA, i.e. the 5nts immediately proximal of the NGG PAM
 #       2-the Tm of the DNA:RNA hybrid from position 8 - 15 (i.e. 8 nt)
 #       3-the Tm of the DNA:RNA hybrid from position 3 - 7  (i.e. 5 nt)
#می خواهیم تک تک ویژگی ها را حساب و تعدادی از آنها را انتخاب و به عنوان ورودی به تابع مربوط به محاسبه اسکور می دهم.
def select_cols(data):
    features = data.loc[:, ['PBSlen', 'RTlen', 'RT-PBSlen', 'Edit_pos', 'Edit_len', 'RHA_len', 'type_sub',
                            'type_ins', 'type_del', 'Tm1', 'Tm2', 'Tm2new', 'Tm3', 'Tm4', 'TmD',
                            'nGCcnt1', 'nGCcnt2', 'nGCcnt3', 'fGCcont1', 'fGCcont2', 'fGCcont3', 'MFE3', 'MFE4', 'DeepSpCas9_score']]

    return features


# برای محاسبه اسکور 3 ورودی می گیرد داده گرفته شده از کاربر، مدل پرایم و نوع سلول
def calculate_deepprime_score(df_input, pe_system='PE2max', cell_type='HEK293T'):

    os.environ['CUDA_VISIBLE_DEVICES']='0'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
#در ادامه کار با مدل برت من جایگزین می شود.
    from genet_models import load_deepprime

    model_dir, model_type = load_deepprime(pe_system, cell_type)

    mean = pd.read_csv('%s/DeepPrime_base/mean.csv' % model_dir, header=None, index_col=0).squeeze()
    std  = pd.read_csv('%s/DeepPrime_base/std.csv' % model_dir, header=None, index_col=0).squeeze()

# در این خط ورودی را از کاربر می گیریم و تمام ویژگی ها را که در ابتدا تعریف کردیم را محاسبه می کنیم و در نهایت می خواهیم به عنوان ورودی به تابع مربوط به محاسبه اسکور بدهیم
    test_features = select_cols(df_input)

    g_test = seq_concat(df_input)

    # این وردی تابع مربوط به محاسبه اسکور است. دقت شود به جای مقادیر اصلی اختلاف آنها با میانگین به مدل داده می شود
    x_test = (test_features - mean) / std

    g_test = torch.tensor(g_test, dtype=torch.float32, device=device)
    x_test = torch.tensor(x_test.to_numpy(), dtype=torch.float32, device=device)


    #  لود کردن مدل ها با توجه به تابعی که برای این کار نوشتم می خواهم چند مدل داشته باشم و از هر کدام خروجی بگیرم و میانگین پاسخ را به خروجی بفرستم
    models = [m_files for m_files in glob('%s/%s/*.pt' % (model_dir, model_type))]
    preds  = []

    for m in models:
        model = GeneInteractionModel(hidden_size=128, num_layers=1).to(device)
        model.load_state_dict(torch.load(m))
        #محاسبه خروجی مدل که همان اسکور نهایی من است
        model.eval()
        with torch.no_grad():
            g, x = g_test, x_test
            g = g.permute((0, 3, 1, 2))
            pred = model(g, x).detach().cpu().numpy()
        preds.append(pred)

    # میانگین پیش بینی ها
    preds = np.squeeze(np.array(preds))
    preds = np.mean(preds, axis=0)
    preds = np.exp(preds) - 1

    return preds

#فراخوانی تابعی که بر اساس مدل و ورودی ها پیش بینی ها را انجام می دهد
#تنها 3 تا از این متغییر ها در حال حاضر از کاربر گرفته می شود و بقیه با مقادیر اولیه مقدار دهی می شود در ادامه بعد از تعریف پوسته سایت این متغییر ها را از کاربر خواهیم گرفت.
# در پایان کد با فراخوانی این تابع اسکورها را محاسبه و چاپ می کنم.
def pe_score(Ref_seq: str,
            ED_seq: str,
            sAlt: str,
            sID:str       = 'Sample',
            pe_system:str = 'PE2max',
            cell_type:str = 'HEK293T',
            pbs_min:int   = 7,
            pbs_max:int   = 15,
            rtt_max:int   = 40
            ):

    nAltIndex   = 60
    pbs_range   = [pbs_min, pbs_max]
    rtt_max     = rtt_max
    pe_system   = pe_system

    edit_type   = sAlt[:-1].lower()
    edit_len    = int(sAlt[-1])

    # check input parameters
    if pbs_max > 17: return print('sID:%s\nPlease set PBS max length upto 17nt' % sID)
    if rtt_max > 40: return print('sID:%s\nPlease set RTT max length upto 40nt' % sID)
    if edit_type not in ['sub', 'ins', 'del']: return print('sID:%s\nPlease select proper edit type.\nAvailable edit tyle: sub, ins, del' % sID)
    if edit_len > 3: return print('sID:%s\nPlease set edit length upto 3nt. Available edit length range: 1~3nt' % sID)
    if edit_len < 1: return print('sID:%s\nPlease set edit length at least 1nt. Available edit length range: 1~3nt' % sID)


#FeatureExtraction Class
# بتدا تمام رشته راهنما های ممکن ساخته می شود
# تابعی را فراخوانی می کنیم تا تمام ویژگی های مربوط به دنباله ها استخراج شود
# سپس اطلاعات مربوط به ساختار دوم
#در نهایت اطلاعات مربوط به اینتراکشن ژن ها
#تابع مربوط به استخراج اطلاعات مربوط به اینتراکشن ژن ها را دارم ولی هنوز نمی دانم چه اطلاعاتی در اسکور دهی پرایم ادیتینگ تاثیر گذار هستند تا آن را به مدل اضافه کنم.

    cFeat = FeatureExtraction()
    cFeat.input_id = sID
    cFeat.get_input(Ref_seq, ED_seq, edit_type, edit_len)

    cFeat.get_sAltNotation(nAltIndex)
    cFeat.get_all_RT_PBS(nAltIndex, nMinPBS=pbs_range[0]-1, nMaxPBS=pbs_range[1], nMaxRT=rtt_max, pe_system=pe_system)
    cFeat.make_rt_pbs_combinations()
    #ویژگی سری 1
    cFeat.determine_seqs()
    #ویژگی سری 2
    cFeat.determine_secondary_structure()
    #ویژگی سری 1
    #تابع را باید بنویسم.


    df = cFeat.make_output_df()

    if len(df) > 0:
      # برای کریسپر طول دنباله را 30 در نظر می گیرم
        list_Guide30 = [WT74[:30] for WT74 in df['WT74_On']]
        # هر دو اسکور اینجا حساب می شود
        df['DeepSpCas9_score'] = spcas9_score(list_Guide30)
        #همه ویژگی هایی را که استخراج کردم را به مدل می دهم
        df['%s_score' % pe_system]  = calculate_deepprime_score(df, pe_system, cell_type)

    else:
        print('\nsID:', sID)
        print('DeepPrime only support RTT length upto 40nt')
        print('There are no available pegRNAs, please check your input sequences\n')

    return df



# برای حالتی که داده ها را از clinVar می گیریم
def pecv_score(cv_record,
               sID:str       = 'Sample',
               pe_system:str = 'PE2max',
               cell_type:str = 'HEK293T',
               pbs_min:int   = 7,
               pbs_max:int   = 15,
               rtt_max:int   = 40
               ):

    '''
    Using variants records from GetClinVar in the database module.\n
    You don't have to bring a sequence input to DeepPrime, but you calculate the score right away.\n
    If DeepPrime is an unpredictable form of variants, it sends out a message.\n

    '''

    # check input parameters
    if pbs_max > 17: return print('sID:%s\nPlease set PBS max length upto 17nt' % sID)
    if rtt_max > 40: return print('sID:%s\nPlease set RTT max length upto 40nt' % sID)

    print('DeepPrime score of ClinVar record')

    Ref_seq, ED_seq = cv_record.seq()

    nAltIndex   = 60
    pbs_range   = [pbs_min, pbs_max]
    rtt_max     = rtt_max
    pe_system   = pe_system

    edit_type   = cv_record.alt_type
    edit_len    = int(cv_record.alt_len)

    ## FeatureExtraction Class
    cFeat = FeatureExtraction()

    cFeat.input_id = sID
    cFeat.get_input(Ref_seq, ED_seq, edit_type, edit_len)

    cFeat.get_sAltNotation(nAltIndex)
    cFeat.get_all_RT_PBS(nAltIndex, nMinPBS=pbs_range[0]-1, nMaxPBS=pbs_range[1], nMaxRT=rtt_max, pe_system=pe_system)
    cFeat.make_rt_pbs_combinations()
    cFeat.determine_seqs()
    cFeat.determine_secondary_structure()

    df = cFeat.make_output_df()

    if len(df) > 0:
        list_Guide30 = [WT74[:30] for WT74 in df['WT74_On']]
        df['DeepSpCas9_score'] = spcas9_score(list_Guide30)
        df['%s_score' % pe_system]  = calculate_deepprime_score(df, pe_system, cell_type)

    else:
        print('\nsID:', sID)
        print('DeepPrime only support RTT length upto 40nt')
        print('There are no available pegRNAs, please check your input sequences\n')

    return df







#کد های کلاس زیر برای کنترل ورودی ها از سایت نوشته شده است.


class DeepPrime:
    '''
    DeepPrime: pegRNA activity prediction models\n
    Input  = 121 nt DNA sequence without edit\n
    Output = 121 nt DNA sequence with edit\n

    ### Available Edit types\n
    sub1, sub2, sub3, ins1, ins2, ins3, del1, del2, del3\n

    ### Available PE systems\n
    PE2, PE2max, PE4max, NRCH_PE2, NRCH_PE2max, NRCH_PE4max\n

    ### Available Cell types\n
    HEK293T, HCT116, MDA-MB-231, HeLa, DLD1, A549, NIH3T3

    '''
    def __init__(self, sID:str, Ref_seq: str, ED_seq: str, edit_type: str, edit_len: int,
                pam:str = 'NGG', pbs_min:int = 7, pbs_max:int = 15,
                rtt_min:int = 0, rtt_max:int = 40, silence:bool = False,
                out_dir:str=os.getcwd(),
                ):

        # input parameters
        self.nAltIndex = 60
        self.sID, self.Ref_seq, self.ED_seq = sID, Ref_seq, ED_seq
        self.edit_type, self.edit_len, self.pam = edit_type, edit_len, pam
        self.pbs_min, self.pbs_max = pbs_min, pbs_max
        self.pbs_range = [pbs_min, pbs_max]
        self.rtt_min, self.rtt_max   = rtt_min, rtt_max
        self.silence = silence

        # output directory
        self.OUT_PATH = '%s/%s/'  % (out_dir, self.sID)
        self.TEMP_DIR = '%s/temp' % self.OUT_PATH

        # initializing
        self.set_logging()
        self.check_input()

        ## FeatureExtraction Class
        cFeat = FeatureExtraction()

        cFeat.input_id = sID
        cFeat.get_input(Ref_seq, ED_seq, edit_type, edit_len)

        cFeat.get_sAltNotation(self.nAltIndex)
        cFeat.get_all_RT_PBS(self.nAltIndex, nMinPBS= self.pbs_min-1, nMaxPBS=self.pbs_max, nMaxRT=rtt_max, pam=self.pam)
        cFeat.make_rt_pbs_combinations()
        cFeat.determine_seqs()
        cFeat.determine_secondary_structure()

        self.features = cFeat.make_output_df()

        del cFeat

        self.logger.info('Created an instance of DeepPrime')

    # def __init__: END


    def submit(self, pe_system:str, cell_type:str = 'HEK293T'):
        print('start pe_scre', self.Ref_seq, self.ED_seq, )

        return None

    # def submit: END


    def set_logging(self):

        self.logger = logging.getLogger(self.OUT_PATH)
        self.logger.setLevel(logging.DEBUG)

        self.formatter = logging.Formatter(
            '%(levelname)-5s @ %(asctime)s:\n\t %(message)s \n',
            datefmt='%a, %d %b %Y %H:%M:%S',
            )

        self.error = self.logger.error
        self.warn  = self.logger.warn
        self.debug = self.logger.debug
        self.info  = self.logger.info

        try:
            os.makedirs(self.OUT_PATH, exist_ok=True)
            os.makedirs(self.TEMP_DIR, exist_ok=True)
            self.info('Creating Folder %s' % self.OUT_PATH)
        except:
            self.error('Creating Folder failed')
            sys.exit(1)

        self.file_handler = logging.FileHandler('%s/log_%s.log' % (self.OUT_PATH, self.sID))
        self.file_handler.setLevel(logging.DEBUG)
        self.file_handler.setFormatter(self.formatter)
        self.logger.addHandler(self.file_handler)

        if self.silence != True:
            self.console_handler = logging.StreamHandler()
            self.console_handler.setLevel(logging.DEBUG)
            self.console_handler.setFormatter(self.formatter)
            self.logger.addHandler(self.console_handler)

        self.info('DeepPrime: pegRNA activity prediction models\n\t version: %s' % genet.__version__)


        return None

    # def set_logging: END


    def check_input(self):

        if self.pbs_min < 1:
            self.error('sID:%s\nPlease set PBS max length at least 1nt' % self.sID)
            raise ValueError('Please check your input: pbs_min')

        if self.pbs_max > 17:
            self.error('sID:%s\nPlease set PBS max length upto 17nt' % self.sID)
            raise ValueError('Please check your input: pbs_max')

        if self.rtt_max > 40:
            self.error('sID:%s\nPlease set RTT max length upto 40nt' % self.sID)
            raise ValueError('Please check your input: rtt_max')

        if self.edit_type not in ['sub', 'ins', 'del']:
            self.error('sID:%s\n\t Please select proper edit type.\n\t Available edit tyle: sub, ins, del' % self.sID)
            raise ValueError('Please check your input: edit_type')

        if self.edit_len > 3:
            self.error('sID:%s\n\t Please set edit length upto 3nt. Available edit length range: 1~3nt' % self.sID)
            raise ValueError('Please check your input: edit_len')

        if self.edit_len < 1:
            self.error('sID:%s\n\t Please set edit length at least 1nt. Available edit length range: 1~3nt' % self.sID)
            raise ValueError('Please check your input: edit_len')

        self.info('Input information\n\t ID: %s\n\t Refseq: %s\n\t EDseq :%s' % (self.sID, self.Ref_seq, self.ED_seq))

        return None

    # def check_input: END


    def do_something(self):
        self.logger.info('Something happened.')

        return None


In [ ]:



#  WT sequence and Edited sequence information,  select the edit type you want to make and put it in.
#Input seq: 60bp 5' context + 1bp center + 60bp 3' context (total 121bp)

seq_wt   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
seq_ed   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
alt_type = 'sub3'

df_pe = prd.pe_score(seq_wt, seq_ed, alt_type)
df_pe.head(10)


[Warnning] genet.predict.pe_score will be deprecated in future.

            Please consider genet.predict.DeepPrime instead.

            Run DeepPrime now anyway.

The model DeepSpCas9 is not installed. Download checkpoint files.



Downloading: 0KB [00:00, ?KB/s]


File downloaded successfully: /usr/local/lib/python3.10/dist-packages/genet/models/DeepSpCas9/__init__.py


Downloading: 681KB [00:00, 4963.17KB/s]                         


File downloaded successfully: /usr/local/lib/python3.10/dist-packages/genet/models/DeepSpCas9/PreTrain-Final-3-5-7-100-70-40-0.001-550-80-60.data-00000-of-00001


...

Downloading: 1KB [00:00, 1324.38KB/s]


File downloaded successfully: /usr/local/lib/python3.10/dist-packages/genet/models/DeepPrime/DP_variant_293T_PE2max_Opti_220428/mean.csv


Downloading: 1KB [00:00, 907.66KB/s]


File downloaded successfully: /usr/local/lib/python3.10/dist-packages/genet/models/DeepPrime/DP_variant_293T_PE2max_Opti_220428/std.csv


,Target,Spacer,RT-PBS,PBSlen,RTlen,RT-PBSlen,Edit_pos,Edit_len,RHA_len,PE2max_score
0,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGG,7,37,44,34,3,1,0.183205
1,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGG,8,37,45,34,3,1,0.567812
2,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT,9,37,46,34,3,1,0.555785
3,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTG,10,37,47,34,3,1,0.880449
4,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGT,11,37,48,34,3,1,0.843617
5,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGTGTT,12,37,49,34,3,1,0.726335
6,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT...,13,37,50,34,3,1,0.878920
7,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT...,14,37,51,34,3,1,0.807085
8,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,TTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGGGT...,15,37,52,34,3,1,0.650620
9,ATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGA...,ATAAAAGACAACACCCTTGCCTTGTGGAGT,GTTCGTCTCAGTTTCTGGGAGCTTTGAAAACTCCACAAGGCAAGG,7,38,45,34,3,2,0.320586


# Utilite

In [ ]:
!pip install RNA

In [ ]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install mechanize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install scikit-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.4 MB/s eta 0:00:00
  Created wheel for scikit-bio: filename=scikit_bio-0.5.9-cp310-cp310-linux_x86_64.whl size=2469054 sha256=4d27b66e6954cd4d0dafa7abf8497c9fc12e46a53205fdea4d9adfdaa67f69a6
  Stored in directory: /root/.cache/pip/wheels/77/72/2c/993efbb4d69a86bee422bc96e4e2f1ec9af7cc596a08bb86e1
  Created wheel for hdmedians: filename=hdmedians-0.14.2-cp310-cp310-linux_x86_64.whl size=677851 sha256=a5fca4892122c67b2bf88210705b2f3338517b253c51406fa87c44f44b6fb961
  Stored in directory: /root/.cache/pip/wheels/82/8f/0d/0c61130cfad119482ebb95ae

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import argparse
import shutil
import datetime
import getpass
import uuid
import mechanize
import re
import pandas as pd
import yaml
import itertools
import pickle
import subprocess
from skbio.alignment import global_pairwise_align_nucleotide
from skbio import DNA
import RNA
from copy import deepcopy as dp
import numpy as np
import sys
import math
#------------------ FILE IO ---------------------------

def write_file(file_name,message):
	out = open(file_name,"wt")
	out.write(message)
	out.close()

# برای کنترل نتایج چاپ شده - می تواند جایگزین تابع بالا شود.
def print_parameters(myDict):
	myGroup = {}
	myGroup['Prime Editing'] = ['genome_fasta','scaffold','n_jobs','debug','PE2_model','PE3_model','extend_length']
	myGroup['PBS searching'] = ['min_PBS_length','max_PBS_length']
	myGroup['RTT searching'] = ['min_RTT_length','max_RTT_length','min_distance_RTT5','max_max_RTT_length']
	myGroup['sgRNA searching'] = ['gRNA_search_space','sgRNA_length','offset','PAM','max_target_to_sgRNA','max_max_target_to_sgRNA']
	myGroup['ngRNA searching'] = ['max_ngRNA_distance']
	for k in myGroup:
		print_group(myDict,myGroup[k],k)


def print_group(myDict,myList,group_title):
	print ("-------- Parameter Group: %s --------"%(group_title))
	for l in myList:
		print ("%s: %s"%(l,myDict[l]))


#تعریف پارامترهای پیش فرض
def get_parameters(config):
	p_dir = os.path.dirname(os.path.realpath(__file__)) + "/"
	# return dict
	parameters = {}
	# default parameters
	pre_defined_list = {}
	#------------ Prime Editing related-----------
	#این ورودی را باید برای برنامه فراهم کنم
	pre_defined_list["genome_fasta"] = "/home/yli11/Data/Human/hg19/fasta/hg19.fa"
	pre_defined_list["n_jobs"] = -1
	pre_defined_list["scaffold"] = "GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC"
	pre_defined_list["debug"] = 0
	pre_defined_list["extend_length"] = 1000 # extracting +- 1000bp center at target pos from the genome, in 99.9% cases, you don't need to change this. If change to less than 500, will trigger fasta input mode, may cause error.
	#   مدل های از پیش آموزش دیده را اینجا معرفی می کنم.
	# می تونی DEEPPRIME بزاری
	pre_defined_list["PE2_model"] = p_dir+"../model/PE2_model_final.py"
	pre_defined_list["PE3_model"] = p_dir+"../model/PE3_model_final.py"

	#------------ PBS -----------
	pre_defined_list["min_PBS_length"] = 10
	pre_defined_list["max_PBS_length"] = 15

	#------------ RTT -----------
	pre_defined_list["min_RTT_length"] = 10
	pre_defined_list["max_RTT_length"] = 20 # if no candidate is found, this value will be increased by 5, max to max_max_RTT_length
	pre_defined_list["max_max_RTT_length"] = 50
	pre_defined_list["min_distance_RTT5"] = 5

	#------------ sgRNA -----------
	pre_defined_list["gRNA_search_space"] = 200
	pre_defined_list["sgRNA_length"] = 20
	pre_defined_list["offset"] = -3
	pre_defined_list["PAM"] = "NGG"
	pre_defined_list["max_target_to_sgRNA"] = 10 # if no candidate is found, this value will be increased by 5, max to max_max_target_to_sgRNA
	pre_defined_list["max_max_target_to_sgRNA"] = 30

	#------------ ngRNA ------------
	pre_defined_list["max_ngRNA_distance"] = 100 # if no candidate is found, this value will be increased by 20, max to max_max_ngRNA_distance
	pre_defined_list["max_max_ngRNA_distance"] = 200
	pre_defined_list["search_iteration"] = 1 # not affect anything

	try:
		with open(config, 'r') as f:
			manifest_data = yaml.load(f,Loader=yaml.FullLoader)
	except:
		print ("Config data is not provided or not parsed successfully, Default parameters were used.")

	for p in pre_defined_list:
		try:
			parameters[p] = manifest_data[p]
		except:
			parameters[p] = pre_defined_list[p]
	return parameters

def to_bed3(chr,start,end):
	outfile = str(uuid.uuid4()).split("-")[-1]
	write_file(outfile,"\t".join([chr,str(start),str(end)]))
	return outfile

def write_fasta(file_name,myDict):
	out = open(file_name,"wt")
	for k in myDict:
		out.write(">"+k+"\n")
		out.write(myDict[k]+"\n")
	out.close()


#طراحی  sgRNA , در ادامه PegRNA
#------------------ sgRNA finder ---------------------------
from Bio import SeqUtils
def run_pam_finder(target_fa,seq,PAM,abs_start_pos,chr):

	# SeqUtils.nt_search("AGGCGGGGG", "NGG")
	# SeqUtils.nt_search("CCACCA", "NGG")
	# forward
	rev_seq = revcomp(target_fa)
	fwd_search = SeqUtils.nt_search(target_fa, seq+PAM)
	rev_search = SeqUtils.nt_search(rev_seq, seq+PAM)
	out = []
	if len(fwd_search) > 1:
		for s in fwd_search[1:]:
			# out.append([chr,s+abs_start_pos,s+abs_start_pos+len(seq),target_fa[s:(s+len(seq))],".","+"])
			out.append([chr,s+abs_start_pos,s+abs_start_pos+len(seq),target_fa[s:(s+len(seq))],target_fa[s:(s+len(seq)+len(PAM))],"+"])
	if len(rev_search) > 1:
		for s in rev_search[1:]:
			# out.append([chr,(len(target_fa)-s)+abs_start_pos-len(seq),(len(target_fa)-s)+abs_start_pos,rev_seq[s:(s+len(seq))],".","-"])
			out.append([chr,(len(target_fa)-s)+abs_start_pos-len(seq),(len(target_fa)-s)+abs_start_pos,rev_seq[s:(s+len(seq))],rev_seq[s:(s+len(seq)+len(PAM))],"-"])
	return pd.DataFrame(out)

def force_recommend_dPAM_PE3b(r,max_eff):
	rank = 0
	if "dPAM" in r.index:
		rank += 1
	if "PE3b" in r.index:
		if not max_eff-r.predicted_efficiency>max_eff*0.1:
			rank += 1
	return rank

#------------------ Fasta Operators ---------------------------
from Bio import SeqIO

def find_pos_ref_alt(x,y):
	"""find variant pos, ref and alt given ref(x) and alt(y) sequences

	Method

	trim 5 end, then trim 3 end, what left is the ref and alt, pos the trim 5 position

	"""

	for i in range(len(x)):
		if x[i]!=y[i]:
			new_x = x[i:]
			new_y = y[i:]
			pos = i+1
			break
	# print (new_x,new_y)
	for i in range(min(len(new_x),len(new_y))):
		i=i+1
		# print ("second for",i,new_x[-i],new_y[-i])
		if new_x[-i]!=new_y[-i]:
			ref = new_x[:-i+1]
			alt = new_y[:-i+1]
			break
		else:

			ref = new_x[:-i]
			alt = new_y[:-i]

	## check
	check_y = x[:pos-1]+alt+x[pos+len(ref)-1:]
	# print ("debug",pos,i,ref)
	# print ("new y",x[:pos-1],x[pos+len(ref)-1:],alt)
	if y!= check_y:
		print ("something is wrong, please fix it")
		print (x)
		print (y)
		return -1,ref,alt
	return pos,ref,alt

def fasta2vcf(f):
	"""convert fasta to vcf dataframe

	Input
	-----

	Fasta file, _ref is recognized as ref and _alt is used as alt, these are two keywords

	Output
	------

	vcf dataframe: chr, pos, name, ref, alt, reference sequence


	"""
	# آنچه یافتم را میخواهم اینجا بریزم
	my_dict = {}
	for r in SeqIO.parse(f, "fasta"):
		my_dict[r.id] = str(r.seq).upper()
	print (my_dict)
	vcf = pd.DataFrame()
	index_list = []
	chr_list = []
	pos_list = []
	ref_list = []
	alt_list = []
	seq_list = []
	for k in my_dict:
		if not "_ref" in k:
			continue
		name = k.replace("_ref","")
		if not name+"_alt" in my_dict:
			print (k,"alt sequence not found. Please use _ref and _alt keywords. Skip...")
			continue
		ref_seq,alt_seq = my_dict[k],my_dict[name+"_alt"]
		if len(ref_seq) < 30:
			print (k,"Please input sequence length at least 30bp. Skip...")
			continue
		if ref_seq == alt_seq:
			print (k,"Ref and Alt sequence is the same. Please check. Skip...")
			continue
		pos,ref,alt = find_pos_ref_alt(ref_seq,alt_seq)
		index_list.append(name)
		chr_list.append(k)
		seq_list.append(ref_seq)
		pos_list.append(pos)
		ref_list.append(ref)
		alt_list.append(alt)
	vcf[0] = chr_list
	vcf[1] = pos_list
	vcf[2] = index_list
	vcf[3] = ref_list
	vcf[4] = alt_list
	vcf[5] = seq_list
	vcf = vcf[vcf[1]!=-1]
	if vcf.shape[0] == 0:
		print ("no valid sequences in:",f)
		print ("Exit...")
		sys.exit(1)

	return vcf

def vcf2fasta(vcf,extend_length=None,genome_fasta=None,**kwargs):
	"""extracting +- extend_length given vcf target mutation

	input
	vcf is a dataframe, chr, pos, id, ref, alt

	return
	--------

	a list of sequences, same order

	"""

	out_bed = str(uuid.uuid4()).split("-")[-1]+".bed"
	out_fa = out_bed+".tab"
	df = vcf.copy()
	df['chr'] = df[0]
	df['start'] = df[1]-extend_length
	df['end'] = df[1]+extend_length
	df[['chr','start','end']].to_csv(out_bed,sep="\t",header=False,index=False)

	p1 = subprocess.Popen(['bedtools','getfasta','-fi',genome_fasta,'-bed',out_bed,'-fo',out_fa,'-tab'],bufsize=0)
	p1.communicate()
	df = pd.read_csv(out_fa,sep="\t",header=None,index_col=0)

	os.remove(out_bed)
	os.remove(out_fa)

	return [x.upper() for x in df[1]]

def get_opposite_strand(x):
	if x == "+":
		return "-"
	return "+"


def sub_fasta_single(target_fa,target_pos, abs_start,abs_end):
	"""given the target_fa we extracted from target_pos, we get sub fasta

	target_fa: we extended +- N bp of the target set, this length is 2N

	user sequence, abs start and end

	target_pos is 1 index, the target pos that we used to get target_fa

	Assumption and user query, target_fa on the same chr

	"""

	N = int(len(target_fa)/2)
	start =  N-(target_pos-abs_start)
	end = abs_end - abs_start + start
	seq = target_fa[start:end]
	if len(target_fa)<1000:
		## user input fasta:
		seq = target_fa[abs_start:abs_end]
	return seq



def get_fasta_simple(target_fa,df, target_pos,strand=False):
	"""save time and memory get fasta

	target_fa: we extended +- N bp of the target set, this length is 2N

	df is a normal bed file

	target_pos is 1 index

	df and target_pos, all on the same chr

	"""
	temp = df.copy()
	# print ("len target_fa",len(target_fa))
	N = int(len(target_fa)/2)
	temp.columns = list(range(len(df.columns)))
	temp.index = temp[0]+":"+temp[1].astype(str)+"-"+temp[2].astype(str)
	if len(target_fa)<1000:
		## user input fasta:
		seq_list = []
		for r in temp.values.tolist():
			seq = target_fa[r[1]:r[2]]
			if strand:
				if r[5] == "-":
					seq = revcomp(seq)
			seq_list.append(seq)
		temp[3] = seq_list
		return temp
	temp[2] = temp[2]-temp[1]
	temp[1] = N-(target_pos-temp[1])
	temp[2] = temp[2]+temp[1]
	seq_list = []
	for r in temp.values.tolist():
		seq = target_fa[r[1]:r[2]]
		if strand:
			if r[5] == "-":
				seq = revcomp(seq)
		seq_list.append(seq)
	temp[3] = seq_list
	return temp



tab = str.maketrans("ACTG", "TGAC")
def revcomp(seq):
	return seq.translate(tab)[::-1]



#------------------ pegRNA Operators ---------------------------


def distance_matrix(lines):
	"""given sgRNA bed dataframe (the 5th is the name), get gRNA distance matrix

	index: chr_start_end_strand_seq (5th column)

	use the last element as the pos to calculate distance

	comparing to D Liu distance, this is always 1 larger than them, because we use the 4th nucleotide as the cut position, cas9 cut between 3rd and 4th

	return
	-------

	2d dict

	"""

	dist_dict={}
	for x in lines:
		dist_dict[x[4]]={}
		for y in lines:
			dist_dict[x[4]][y[4]] = x[-1]-y[-1]
	return dist_dict



def is_gRNA_valid(cas9_cut_position,target_mutation,strand,user_target_mutation_pos,diff):
	# cas9_cut_position=[chr,pos], pos is 1-based position, same as in vcf file
	# target_mutation=[chr,pos], pos is 1-based position, same as in vcf file
	"""
	PBS sequence can't be on the same side to the target site in terms of the cut site

	The position of the target mutation should be:
	On the right of the cas9 cut position if gRNA strand is +
	On the left of the cas9 cut position if gRNA strand is -

	user_target_mutation_pos, mutation correct cause bug when strand = -

	Return
	------

	is gRNA valid, target mutation distance to cut site


	"""
	if cas9_cut_position[0] != target_mutation[0]:
		return -1
	distance = int(target_mutation[1]-cas9_cut_position[1])
	# print (cas9_cut_position,target_mutation,strand,user_target_mutation_pos,diff,distance)
	if strand=="+":
		if distance>=0:
			return distance
	if strand=="-":
		if distance == 0:
			if user_target_mutation_pos != target_mutation[1]:
				distance = -1
		if distance<=0:
			distance = -distance
			if diff > 0:
				distance  = distance - diff + 1
			return distance
	return -1




def get_gRNA_cut_site(start,end,strand,offset=-3):
	# return 1-index pos
	# cut site = the first base before cut near PAM direction

	if strand == "+":
		return int(end + offset)
	if strand == "-":
		return int(start - offset +1)



#------------------ featurize ---------------------------


def GC_content(seq):
	GC=["G","C"]
	count=0
	for i in seq:
		if i in GC:
			count+=1
	return float(count)/len(seq)
	pass



ulength = 31
# ViennaRNAاز کتابخانه
md = RNA.md()
md.max_bp_span = 70
md.window_size = 70

def call_RNAplfold(seq,scaffold_length):
	"""RNA fold - binding
	"""
	# scaffold - RTT - PBS
	# we only care about RTT+PBS (3' extension) pairs with scaffold

	data = []
	fc = RNA.fold_compound(seq, md, RNA.OPTION_WINDOW)
	fc.probs_window(ulength, RNA.PROBS_WINDOW_BPP | RNA.PROBS_WINDOW_UP, pf_window_callback2, data)

	# parse output
	df = pd.DataFrame(data)
	df2 = df.copy()
	df2[0] = df[1]
	df2[1] = df[0]

	#
	df = pd.concat([df,df2])
	seq_length = 10
	RTT_start = scaffold_length + 1
	RTT_end = scaffold_length + seq_length
	scaffold_start = 1
	scaffold_end =  scaffold_length

	## subset df
	df = df[df[0]>=RTT_start]
	df = df[df[0]<=RTT_end]
	df = df[df[1]>=scaffold_start]
	df = df[df[1]<=scaffold_end]
	df = df[[0,2]]
	df = df.groupby(0).max()
	myDict = df[2].to_dict()

	value_list = []
	for i in range(seq_length):
		RTT_pos = scaffold_length +i+ 1
		if RTT_pos in myDict:
			value_list.append(myDict[RTT_pos])
		else:
			value_list.append(0)
	return value_list


def pf_window_callback2(v, v_size, i, maxsize, what, data=None):
	if what & RNA.PROBS_WINDOW_UP:
		pass
	else:
		data+=[[i,j,p] for j, p in enumerate(v) if (p is not None) and (p >= 0.01)]

def local_alignments(ref,q):
	query = StripedSmithWaterman(ref)
	alignment = query(q)
	return alignment['optimal_alignment_score']


def alignments_to_cigar(ref,q):
	Match = 0
	Mis = 0
	D = 0
	I = 0
	for i in range(len(ref)):
		a=ref[i]
		b=q[i]
		if a=="-":
			I=I+1
		elif b=="-":
			D=D+1
		elif a==b:
			Match+=1
		else:
			Mis+=1
	return [Match,Mis,D,I]

def global_alignments(ref,q):

	s1 = DNA(ref)
	s2 = DNA(q)
	alignment, score, start_end_positions = global_pairwise_align_nucleotide(s1,s2,match_score=4,mismatch_score=1)
	return alignments_to_cigar(alignment[0]._string.decode("utf-8"),alignment[1]._string.decode("utf-8"))

def is_dPAM(PAM_seq, RTT, cut_offset=-3):
	# Assuming no N is RTT, which should be true
	# match PAM seq to RTT, should be abs(cut_offset)
	# print (PAM_seq, RTT)
	# will need to do revcomp no matter what, because RTT is always xxxxxxxPAM

	seq = revcomp(RTT)
	fwd_search = SeqUtils.nt_search(seq, PAM_seq)
	flag = 1
	if len(fwd_search) > 1:
		if abs(cut_offset) in fwd_search:
			flag = 0

	return flag

def target_to_RTT5_feature(pegRNA,nick_gRNA,target_loc,RTS_length,alt_length):
	# 1. target mutation distance to cut 1 (pegRNA)
	# 2. target mutation distance to cut 2 (nick-gRNA)
	# target_loc [chr,pos]
	# 3. cut 1 to cut 2
	cut1 = get_gRNA_cut_site(pegRNA[1],pegRNA[2],pegRNA[3])
	## if nick_gRNA not exist return a big number
	if nick_gRNA[0] == "0":
		cut2=0
	else:
		cut2 = get_gRNA_cut_site(nick_gRNA[1],nick_gRNA[2],nick_gRNA[3])

	# cut2 to cut1
	a=cut2-cut1-1


	# target to cut1
	b=target_loc[1]-cut1

	if pegRNA[3]=="-":
		a+=2 # match to coordinate system
		a=-a
		b=-b
	c=cut2-target_loc[1]
	if nick_gRNA[0] == "0":
		a=np.nan
		c=np.nan
	if b <0:
		print ("pegRNA to target is less than 0!")
		exit()
	d = RTS_length - alt_length- b + 1 ## number of nucleotide to the RTT 5' end, from the target mutation (not including)
	if d <0:
		print ("pegRNA to target is less than 0!")
		exit()
	index_list = ["nick_to_pegRNA","target_to_pegRNA","target_to_ngRNA","target_to_RTT5"]
	out = pd.DataFrame([a,b,c,d])
	out.index = index_list
	return out


	pass


#------------------ DeepSpCas9 score  ---------------------------
#قبلا در بالا مدل را دانملود و استفاده میکنم

def list_to_fasta(l):
	out = []
	for i in l:
		if len(i) != 23:
			print ("something is wrong")
		out.append(">%s"%(i[:20]))
		out.append("AAAA%sAAA"%(i))
	return "\n".join(out)
def parse_webpage(c):
	a=re.findall('"([^"]*)"', str(c))
	# get the random id
	for x in a:
		# random id format could be changed during version updates
		if "DeepSpCas9/job/user" in x:
			random_id = x.split("/")[-1]
	url = "http://deepcrispr.info/DeepSpCas9/data/%s/Results.zip"%(random_id)
	df = pd.read_csv(url,sep="\t")
	df[df.ID==df['Guide Sequence (20bp)']]
	df.index = df.ID.tolist()
	return df['DeepSpCas9 Score'].to_dict()

def get_DeepSpCas9_score(gRNA_list):
	"""Grab score from web server

	input gRNA list should include PAM sequence, PAM is NGG
	"""
	url="http://deepcrispr.info/DeepSpCas9/"
	br = mechanize.Browser()
	br.set_handle_robots(False) # ignore robots
	br.open(url)
	br.select_form(nr=0)
	br["ENTER_FASTA"] = list_to_fasta(gRNA_list)
	res = br.submit()
	output = res.read()
	res = parse_webpage(output)
	flag = False
	for i in gRNA_list:
		if not i[:20] in res:
			print ("gRNA: %s NOT FOUND!"%(i[:20]))
			print ("DeepSpCas9 API error!")
			flag = True
			res[i[:20]] = 0
	if flag:
		print (output)
		print (res)
	return res
# gRNA_list = ['GGAATCCCTTCTGCAGCACCAGG','GGCCCAGACTGAGCACGTGAAGG']
# res = get_DeepSpCas9_score(gRNA_list)
# GGAATCCCTTCTGCAGCACC    55.750
# GGCCCAGACTGAGCACGTGA    52.897
# Name: DeepSpCas9 Score, dtype: float64

Utilities 2

---



In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import dash_table
from flask import Flask, send_from_directory
from urllib.parse import quote as urlquote
import dash_bootstrap_components as dbc

# import plotly_express as px
from dash.dependencies import Input, Output, State
import re
import os
import numpy as np
from sklearn.cluster import KMeans
import plotly.figure_factory as ff
import os
import base64
import datetime
from datetime import datetime as dt
import pathlib
import easy_prime
import io
import csv
from easy_prime.utils import get_parameters, print_parameters,vcf2fasta,fasta2vcf
from easy_prime import target_mutation
import subprocess
from joblib import Parallel, delayed
import urllib
from io import StringIO
import uuid
from Bio import SeqIO
import math
import pandas as pd
import uuid
import subprocess
import matplotlib.pyplot as plt
npg_colors = ["#E64B35","#4DBBD5","#00A087","#3C5488","#F39B7F","#464d4f"]
my_colors = {}
my_colors['sgRNA'] = npg_colors[0]
my_colors['PBS'] = npg_colors[1]
my_colors['RTT'] = npg_colors[2]
my_colors['ngRNA'] = npg_colors[3]
my_colors['variant'] = "#e6fc3f"


def get_version():

	return "v%s"%(easy_prime.__version__)

def vis_pegRNA_png(df_file,jid):
	# bedtools 2.29.2
	# print ("call easy_prime vis")
	genome_fasta = "/home/yli11/Data/Human/hg19/fasta/hg19.fa"
	if os.path.isfile(f"results/{jid}.fa"):
		subprocess.call(f"easy_prime_vis -f {df_file} -s results/{jid}.fa --output_file_name results/{jid}.png",shell=True)
	else:
		# print ("using genome fasta")
		cmd = f"easy_prime_vis -f {df_file} -s {genome_fasta} --output_file_name results/{jid}.png"
		# print (cmd)
		subprocess.call(cmd,shell=True)
	fig = f"results/{jid}.png"
	with open(fig, "rb") as image_file:
		img_string = base64.b64encode(image_file.read())
	return "data:image/png;base64,%s"%(img_string.decode("utf-8"))


def df2csv_string(df):
	csv_string = df.to_csv(index=False, encoding='utf-8')
	csv_string = "data:text/csv;charset=utf-8,%EF%BB%BF" + urllib.parse.quote(csv_string)
	return csv_string

def file_download_link(filename):
    """Create a Plotly Dash 'A' element that downloads a file from the app."""
    location = "results/{}".format(urlquote(filename))
    return html.A(filename, href=location,target="_blank")

def get_current_pegRNA_table_title_and_download_links(df,jid):
	PE = "%.1f"%(df.predicted_efficiency[0])+"%"
	use_columns = ['chr','start','end','seq',"type",'strand']
	show_columns = ['#chr','start','end','seq',"type",'strand']
	df = df[use_columns]
	df.columns = show_columns
	table =  dash_table.DataTable(
		id='pegRNA-table',
		columns=[
			{'name': i, 'id': i, 'deletable': False} for i in show_columns
		],
		data=df.to_dict('records'),
	)

	header = dbc.FormGroup(
		[
			html.H5("Current pegRNA/ngRNA selection. Predicted efficiency: %s"%(PE),style={"margin-right":10,"margin-left":10}),
			html.A(html.Button('Download current selection',className="btn btn-dark"),href=df2csv_string(df[show_columns]),target="_blank",download="current_design.csv" ,style={"margin-right":10}),
			html.A(html.Button('Download all predictions',className="btn btn-dark"),href="results/{}".format(urlquote("%s_rawX_pegRNAs.csv.gz"%(jid))),target="_blank",style={"margin-right":10}),
		],
		row=True,
	)
	return [header,table]

def get_current_selection_table(df):
	show_columns = ['chr','start','end','seq',"predicted_efficiency",'strand']
	table =  dash_table.DataTable(
		id='pegRNA-table',
		columns=[
			{'name': i, 'id': i, 'deletable': False} for i in show_columns
		],
		data=df.to_dict('records'),
	)

	return [html.H5("Current pegRNA/ngRNA selection"),table]


download_current_selection_button = html.A(html.Button('Submit feedback!'),href='https://github.com/czbiohub/singlecell-dash/issues/new',target="_blank")


#---------------------------------  prime search ---------------------------
def write_fasta(file_name,myDict):
	out = open(file_name,"wt")
	for k in myDict:
		out.write(">"+k+"\n")
		out.write(myDict[k]+"\n")
	out.close()
def read_fasta(f):
	my_dict = {}
	for r in SeqIO.parse(f, "fasta"):
		my_dict[r.id] = str(r.seq).upper()
	return my_dict
def run_steps(t,**kwargs):

	t.init(**kwargs)
	t.search(**kwargs)
	t.predict(**kwargs)

	return [t.topX,t.rawX,t.X_p,t.found_PE3b,t.found_PE3,t.found_dPAM,t.found_PE2,t.N_sgRNA_found]

def run_easy_prime_backend():

	error_message = ""
	return False,error_message

def run_easy_prime_backend(vcf,jid,parameters):
	# print (vcf.head())
	if vcf.shape[1]==5:
		vcf[5] = vcf2fasta(vcf,**parameters)
		vcf = vcf[list(range(6))]

	variant_list = vcf[2].tolist()
	my_targets = [target_mutation(*r) for i,r in vcf.iterrows()]

	df_list = [run_steps(t,**parameters) for t in my_targets]

	summary = pd.DataFrame([x[3:8] for x in df_list]).astype(int)
	summary.columns = ['found_PE3b','found_PE3','found_dPAM','found_PE2',"N_sgRNA_found"]
	summary.index = variant_list
	summary.to_csv("results/%s_summary.csv"%(jid),index=True)

	df_top = pd.concat([x[0] for x in df_list])
	if df_top.shape[0]==0:
		# print ("no pegRNA (including PE2) were found for the input file")
		return summary,pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
	df_top = df_top.sort_values("predicted_efficiency",ascending=False)
	df_top.to_csv("results/%s_topX_pegRNAs.csv"%(jid),index=False)
	df_all = pd.concat([x[1] for x in df_list])
	df_all = df_all.sort_values("predicted_efficiency",ascending=False)
	df_all.to_csv("results/%s_rawX_pegRNAs.csv.gz"%(jid),index=False,compression="gzip")

	X_p = pd.concat([x[2] for x in df_list])
	X_p = X_p.sort_values("predicted_efficiency",ascending=False)
	X_p.to_csv("results/%s_X_p_pegRNAs.csv.gz"%(jid),index=True,compression="gzip")
	return summary,df_top,df_all,X_p

def PD2fasta_dict(input_string):
	out = {}
	myDict = read_fasta(StringIO(input_string))
	for k in myDict:
		s = myDict[k]
		if s.count("(")>1:
			return 0
		before = s[:s.find("(")]
		after = s[s.find(")")+1:]
		mutation = s[s.find("(")+1:s.find(")")]
		if "+" in mutation:
			mutation = mutation.replace("+","")
			ref = before+after
			alt = before+mutation+after
		elif "-" in mutation:
			mutation = mutation.replace("-","")
			ref = before+mutation+after
			alt = before+after
		else:
			mutation = mutation.split("/")
			ref = before+mutation[0]+after
			alt = before+mutation[1]+after
		out["%s_ref"%(k)] = ref
		out["%s_alt"%(k)] = alt
	return out
main_chr_human = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chrX","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr20","chrY","chr19","chr22","chr21","chrM"]

def is_valid_DNA(sequence):
	valid_dna = "ACGT"
	return all(i in valid_dna for i in sequence)

def check_and_convert_input(input_type,chr,pos,variant_id,ref,alt,vcf_batch,fasta_input,PrimeDesign_input,jid):
	error_flag = False
	error_message = ""
	if input_type == "vcf_tab":
		if not "chr" in chr:
			chr = "chr"+chr
		if not chr in main_chr_human:
			return None, True,"Input chromosome %s not found in hg19 main chromosomes"%(chr)
		try:
			pos = int(pos)
		except:
			return None, True,"Input Position %s can't converted to integer"%(pos)
		name = variant_id
		if name == "":
			return None, True,"Input Name is empty!"
		ref = ref.upper()
		if ref == "":
			return None, True,"Input Name is empty!"
		alt = alt.upper()
		if alt == "":
			return None, True,"Input Name is empty!"
		if not is_valid_DNA(ref):
			return None, True,"Input reference allele is not valid, only A,C,G,T is allowed!"
		if not is_valid_DNA(alt):
			return None, True,"Input alternative allele is not valid, only A,C,G,T is allowed!"
		vcf = pd.DataFrame([[chr,pos,name,ref,alt]])

	if input_type == "vcf_batch_tab":
		try:
			vcf_batch = vcf_batch.replace(" ","\t")
			vcf = pd.read_csv(StringIO(vcf_batch),comment="#",sep="\t",header=None)
			vcf[1] = vcf[1].astype(int)
			vcf =vcf.drop_duplicates(2) # remove duplicated names
			vcf[3] = [x.upper() for x in vcf[3]]
			vcf[4] = [x.upper() for x in vcf[4]]
		except Exception as e:
			# print (e)
			return None, True,"Input vcf_batch can't be parsed correctly! %s"%(e)
	if input_type == "fasta_tab":
		try:
			file_name = "results/%s.fa"%(jid)
			myDict = read_fasta(StringIO(fasta_input))
			write_fasta(file_name,myDict)
			vcf = fasta2vcf(file_name)
		except Exception as e:
			# print (e)
			return None,True,"Input fasta can't be parsed correctly! Make sure fasta length >= 50bp. Error: %s"%(e)
	if input_type == "PrimeDesign_tab":
		try:
			file_name = "results/%s.fa"%(jid)
			myDict = PD2fasta_dict(PrimeDesign_input)
			write_fasta(file_name,myDict)
			vcf = fasta2vcf(file_name)
		except Exception as e:
			# print (e)
			return None,True,"Input PrimeDesign sequences can't be parsed correctly! Currently, combinatorial editing in this format is not supported! %s"%(e)


	return vcf,False,error_message



def read_easy_prime_output(jid):
	rawX = pd.read_csv("results/%s_rawX_pegRNAs.csv.gz"%(jid))
	X_p = pd.read_csv("results/%s_X_p_pegRNAs.csv.gz"%(jid),index_col=0)
	X_p['sample_ID'] = X_p.index.tolist()
	rawX.index = rawX.sample_ID.tolist()

	rawX['location_name'] = rawX.chr+"_"+rawX.start.astype(str)+"_"+rawX.end.astype(str)+"_"+rawX.seq
	rawX['DeepSpCas9_score'] = X_p['cas9_score']
	rawX['target_pos'] = X_p['Target_pos']
	rawX['nick_pos'] = X_p['nick_to_pegRNA']
	rawX['PBS_length'] = X_p['PBS_length']
	rawX['RTT_length'] = X_p['RTT_length']
	rawX['PE3b'] = X_p['PE3b']
	return rawX,X_p

def get_options_dict(jid):
	df = pd.read_csv("results/%s_summary.csv"%(jid),index_col=0)
	# print (df.head())
	out = []
	first_valid = None
	for i,r in df.iterrows():
		if r.found_PE2 == 0:
			out.append({'label': i + " | no pegRNA found",'disabled': True, 'value': i})
		else:
			if not first_valid:
				first_valid = i
			out.append({'label': i, 'value': i})
	return out,first_valid

def get_annotation_dPAM(x):
	if "dPAM" in x:
		return "PAM-disruption"
	return ""

def get_annotation_PE3b(x):

	if "PE3b" in x:
		return "PE3b"
	return ""
def force_recommend_dPAM_PE3b(r,max_eff):
	rank = 0
	if "PE3b" in r.sample_ID:
		if not max_eff-r.predicted_efficiency>max_eff*0.1:
			rank += 1
	if "dPAM" in r.sample_ID:
		rank += 1
	return rank
def to_sgRNA_table(rawX,sample_ID):
	rawX_df = rawX[rawX.sample_ID.str.contains(sample_ID)]
	rawX_df = rawX_df[rawX_df['type']=='sgRNA']
	# X_p_df = X_p[X_p.sample_ID.str.contains(sample_ID)]
	rawX_df = rawX_df.sort_values('predicted_efficiency',ascending=False)
	rawX_df = rawX_df.drop_duplicates(['chr','start','end'])
	columns = ['chr','start','end','seq','DeepSpCas9_score','strand','target_pos','annotation']
	# tmp = rawX_df.copy()
	rawX_df['rank'] = rawX_df.apply(lambda r:force_recommend_dPAM_PE3b(r,rawX_df.predicted_efficiency.max()),axis=1)
	rawX_df = rawX_df.sort_values(['rank','predicted_efficiency'],ascending=False)
	rawX_df['annotation'] = rawX_df.sample_ID.apply(get_annotation_dPAM)

	rawX_df = rawX_df.reset_index(drop=True)
	return rawX_df[columns]

def to_sgRNA_table2(rawX,X_p,sample_ID):
	rawX_df = rawX[rawX.sample_ID.str.contains(sample_ID)]
	rawX_df = rawX_df[rawX_df['type']=='sgRNA']
	X_p_df = X_p[X_p.sample_ID.str.contains(sample_ID)]
	rawX_df = rawX_df.sort_values('predicted_efficiency',ascending=False)
	rawX_df = rawX_df.drop_duplicates(['chr','start','end'])

	columns = ['chr','start','end','seq','DeepSpCas9_score','strand','target_pos','annotation']
	rawX_df['DeepSpCas9_score'] = X_p_df['cas9_score']
	rawX_df['target_pos'] = X_p_df['Target_pos']
	rawX_df['annotation'] = ""
	rawX_df = rawX_df.reset_index(drop=True)
	ID_list = rawX_df.sample_ID.tolist()
	return rawX_df[columns],ID_list


def to_ngRNA_table(rawX,sgRNA_location):
	sample_ID_list = rawX[(rawX.location_name==sgRNA_location)&(rawX['type']=='sgRNA')].sample_ID.tolist()
	rawX_df = rawX[rawX.sample_ID.isin(sample_ID_list)]
	rawX_df = rawX_df[rawX_df['type']=='ngRNA']
	rawX_df = rawX_df.drop_duplicates(['chr','start','end'])
	columns = ['chr','start','end','seq','nick_pos','strand','annotation']
	rawX_df['annotation'] = rawX_df.sample_ID.apply(get_annotation_PE3b)

	rawX_df['rank'] = rawX_df.apply(lambda r:force_recommend_dPAM_PE3b(r,rawX_df.predicted_efficiency.max()),axis=1)
	# print (rawX_df.predicted_efficiency.max())
	# rawX_df = rawX_df.sort_values(['rank','predicted_efficiency'],ascending=False)
	# print (rawX_df['rank'].unique())

	rawX_df['nick_pos_abs'] = rawX_df['nick_pos'].abs()
	rawX_df = rawX_df.sort_values('nick_pos_abs')
	rawX_df = rawX_df.drop(['nick_pos_abs'],axis=1)
	rawX_df = rawX_df.reset_index(drop=True)
	# print (rawX_df.head())

	return rawX_df[columns],rawX_df.sort_values(['rank','predicted_efficiency'],ascending=False).index[0]


def to_PBS_table(rawX,sgRNA_location):
	sample_ID_list = rawX[(rawX.location_name==sgRNA_location)&(rawX['type']=='sgRNA')].sample_ID.tolist()
	rawX_df = rawX[rawX.sample_ID.isin(sample_ID_list)]
	rawX_df = rawX_df[rawX_df['type']=='PBS']
	columns = ['chr','start','end','seq','PBS_length','strand']
	rawX_df = rawX_df.drop_duplicates('seq')

	rawX_df['rank'] = rawX_df.apply(lambda r:force_recommend_dPAM_PE3b(r,rawX_df.predicted_efficiency.max()),axis=1)
	# rawX_df['PBS_length'] = rawX_df.seq.apply(len)
	rawX_df = rawX_df.sort_values('PBS_length')
	rawX_df = rawX_df.reset_index(drop=True)

	return rawX_df[columns],rawX_df.sort_values(['rank','predicted_efficiency'],ascending=False).index[0]


def to_RTT_table(rawX,sgRNA_location):
	sample_ID_list = rawX[(rawX.location_name==sgRNA_location)&(rawX['type']=='sgRNA')].sample_ID.tolist()
	rawX_df = rawX[rawX.sample_ID.isin(sample_ID_list)]
	rawX_df = rawX_df[rawX_df['type']=='RTT']
	columns = ['chr','start','end','seq','RTT_length','strand']
	rawX_df = rawX_df.drop_duplicates('seq')
	# rawX_df['RTT_length'] = rawX_df.seq.apply(len)
	rawX_df['rank'] = rawX_df.apply(lambda r:force_recommend_dPAM_PE3b(r,rawX_df.predicted_efficiency.max()),axis=1)
	rawX_df = rawX_df.sort_values('RTT_length')
	rawX_df = rawX_df.reset_index(drop=True)

	return rawX_df[columns],rawX_df.sort_values(['rank','predicted_efficiency'],ascending=False).index[0]



def to_RTT_table2(rawX,sample_ID_list,best_ID=None):
	if not best_ID:
		best_ID = sample_ID_list[0]
	rawX_df = rawX[rawX.sample_ID.isin(sample_ID_list)]
	rawX_df = rawX_df[rawX_df['type']=='RTT']
	columns = ['chr','start','end','seq','RTT_length','strand']
	rawX_df = rawX_df.drop_duplicates('seq')
	rawX_df['RTT_length'] = rawX_df.seq.apply(len)
	rawX_df = rawX_df.sort_values('RTT_length')
	rawX_df = rawX_df.reset_index(drop=True)
	return rawX_df[columns],rawX_df[rawX_df.sample_ID == best_ID].index.tolist()


def get_uid():
	# return "easy_prime_yli11_2021-05-24_result_dir"
	return str(uuid.uuid4()).split("-")[-1]




#--------------------------------- dash app utils ---------------------------


def df2bedjs(df,output):
	# sample_ID,CHROM,POS,REF,ALT,type,seq,chr,start,end,strand,predicted_efficiency
	# FIG5G_HEK293T_HEK3_6XHIS_chr9_110184619_110184639_+_GGCCCAGACTGAGCACGTGA_candidate_1808,chr9,110184636,G,GCACCATCATCACCATCAT,ngRNA,GTCAACCAGTATCCCGGTGC,chr9,110184723,110184743,-,0.6275171041488647
	df['name'] = df.apply(lambda r:"""{"strand":"%s","name":"%s","color":"%s"}"""%(r.strand,r['type'],my_colors[r['type']]),axis=1)
	track_name = "pegRNA_design_%.1f"%(df.predicted_efficiency[0])
	df = df[['chr','start','end','name']]
	# print (df.head())
	df.sort_values('start').to_csv("results/%s.bed"%(output),sep="\t",header=False,index=False,quoting=csv.QUOTE_NONE)
	os.system("bgzip -f results/{0}.bed;tabix -f -p bed results/{0}.bed.gz".format(output))

	return '''{"type":"bedj","url":"http://easy-prime-test-dev.us-west-2.elasticbeanstalk.com/results/%s.bed.gz","stackheight":20,"stackspace":1,"name":"%s"},'''%(output,track_name)


ModuleNotFoundError: ignored

# PegRNA

PegRNA

In [ ]:
class sgRNA:
	def __init__(self,chr=None,start=None,end=None,seq=None,sgRNA_name=None,strand=None,cut_position=None,mutation_pos = None,mutation_ref = None,mutation_alt = None,variant_id=None,dist_dict=None,opposite_strand_sgRNAs=None,all_sgRNA_df=None,target_fa=None,scaffold_seq=None,user_target_pos=None,user_ref=None,user_alt=None,offset=None,target_to_sgRNA=None,PAM=None,DeepSpCas9=None,**kwargs):

		#search key is chr_start_end_seq because seq can be duplicated, this chr_start_end_seq is unique. This name doesn't show strand, but seq is alwasy 5-3 direction
		#Searching steps
		#1. RTT
		#2. PBS
		#3. ngRNA
	#	---------------------------------------------------------------------------------------------------------------------------------

	# مقدار دهی متغییر ها با مقادیر ارسال شده به کلاس
		self.chr = chr
		self.start = start
		self.end = end
		self.seq = seq
		self.variant_id = variant_id
		self.target_to_sgRNA = target_to_sgRNA
		self.DeepSpCas9 = DeepSpCas9
		self.is_dPAM = 0
		self.PAM = PAM
		self.offset = offset
		self.sgRNA_name = sgRNA_name
		self.uid = str(uuid.uuid4()).split("-")[-1]
		# self.name = "_".join([chr,str(start),str(end),seq])
		self.strand = strand
		self.cut_position = cut_position
		print (cut_position)
		# exit()
		self.target_pos = mutation_pos ## this is the corrected, actual position of variant
		self.target_fa = target_fa
		self.scaffold_seq = scaffold_seq
		# self.max_nick_distance = max_nick_distance
		self.ref = mutation_ref
		self.alt = mutation_alt
		self.dist_dict = dist_dict ## other gRNAs in search space
		self.opposite_strand_sgRNAs = opposite_strand_sgRNAs
		# self.candidate_pegRNA_df = candidate_pegRNA_df	 ## other gRNAs in search space
		self.nick_gRNA_list = []
		self.user_target_pos = user_target_pos
		self.user_ref = user_ref
		self.user_alt = user_alt



		#for rawX
		#		Searching steps
		#1. RTT
		#2. PBS
		#3. ngRNA
#------------------------------------------------------------------
		#برای فهم بهتر ویژگی های هر سه دنباله را از هم جدا کردم و برای هر یم فریم دیتای جداگانه ساختم.
		#ویژگی های تاثیرگذار برای هر دنباله در اینجا مشخص می شود

		self.PBS_df = pd.DataFrame()
		self.RTT_df = pd.DataFrame()
		self.ngRNA_df = pd.DataFrame()
		self.PBS_feature_list = ["PBS_GC",'PBS_length']
		#بعد اینکه ویژگی تاثیرگذار را یافتم نوکلئوتیدهای آخر را حذف کردم
		# self.RTT_feature_list = ["target_to_RTT5","RTT_GC","RTT_length","0","1","2","3","4","5","6","7","8","9","10","11","12","13"]
		self.RTT_feature_list = ["target_to_RTT5","RTT_GC","RTT_length","0","1","2","3","4","5","6","7","8","9"]
		self.ngRNA_feature_list = ['sgRNA_distance_to_ngRNA','is_PE3b']
		self.rawX = pd.DataFrame()
		self.X = pd.DataFrame()
		self.X_p = pd.DataFrame()
		# sgRNA name = chr,start,end,seq,strand
		## flags
		self.no_RTT = False ## alwasy assume we can find valid RTT
		self.no_ngRNA = False ## alwasy assume we can find valid ngRNA





#--------------------------------------------------------------------------------------------------------
#  PBS sequences هدف این تابع پیدا کردن
# find PBS sequences as bed format df
#		:خروجی های این تابع
#		1.PBS_df
#		2.PBS_feature_list
#لیست و دیتا فریم را در ابتدای کلاس تعریف  کرده بودم
	def find_PBS(self,min_PBS_length=7,max_PBS_length=17,**kwargs):

    #این فلگ در بالا به صورت پیش فرض 0 در نظر گرفته شده است
		if self.no_RTT:
			return 0

		out = []
		chr = self.chr

		if self.strand=="+":
			end = self.cut_position
			for l in range(min_PBS_length,max_PBS_length+1):
				start = end-l
				out.append([chr,start,end])
		if self.strand=="-":
			start = self.cut_position - 1
			for l in range(min_PBS_length,max_PBS_length+1):
				end = start+l
				out.append([chr,start,end])
		self.PBS_df = pd.DataFrame(out)
		self.PBS_df.columns = ['chr','start','end']
		self.PBS_df["strand"] = get_opposite_strand(self.strand)
		self.PBS_df.index = ["%s_PBS_%s"%(self.uid,i) for i in range(self.PBS_df.shape[0])]
		temp = get_fasta_simple(self.target_fa,self.PBS_df, self.user_target_pos)
		self.PBS_df['seq'] = temp[3].tolist()
    ## when sgRNA is positive strand, RTT should use the negative strand
		if self.strand == "+":
			self.PBS_df['seq'] = [revcomp(x) for x in self.PBS_df['seq']]

		self.PBS_df['PBS_GC'] = [GC_content(x) for x in self.PBS_df['seq'] ]
		self.PBS_df['PBS_length'] = [len(x) for x in self.PBS_df['seq'] ]
		print(self.PBS_df)




#---------------------------------------------------------------------------------------------
	# find RTT sequences as bed format df
	#	Output:
	#1. RTT_df
	#2. RTT_feature_list

	def find_RTT(self,debug=0,min_RTT_length=10,max_RTT_length=20,max_max_RTT_length=40,min_distance_RTT5=5,**kwargs):

		out = []
		chr = self.chr
		pbs_start = None
		pbs_end = None
		user_max_RTT_length = max_RTT_length
		large_deletion_flag=False

		if len(self.ref)+min_distance_RTT5 > max_RTT_length: ## in case of large deletion
			large_deletion_flag = True
			max_RTT_length = max_RTT_length+len(self.ref)
		# target_to_RTT5_feature=[]
		deletion_length = len(self.ref)-len(self.alt)
		if self.strand=="+":
			start = self.cut_position # remember out cut position, the actual nucleotide, we use -4
			pbs_end = start
			pbs_start = pbs_end - 14
			for l in range(min_RTT_length,max_RTT_length+1+max(0,deletion_length)):
				end = start+l
				if start+1<=self.target_pos <=end-min_distance_RTT5:
					out.append([chr,start,end,end-self.target_pos-max(0,deletion_length)])
		if self.strand=="-":
			end = self.cut_position - 1
			pbs_start = end
			pbs_end = pbs_start + 14
			for l in range(min_RTT_length,max_RTT_length+1+max(0,deletion_length)):
				start = end-l
				if end>=self.target_pos >=start+1+min_distance_RTT5:
					out.append([chr,start,end,self.target_pos-start-1-max(0,deletion_length)])

		current_max_RTT_length = max_RTT_length+5
		while len(out) == 0:
			if current_max_RTT_length > max_max_RTT_length:
				break
			out,_,no_RTT_flag = self.find_longer_RTT(min_RTT_length=min_RTT_length,max_RTT_length=current_max_RTT_length,min_distance_RTT5=min_distance_RTT5,**kwargs)
			if len(out) > 0:
				print ("max_RTT_length increased from %s to %s"%(max_RTT_length,current_max_RTT_length))
				break
			current_max_RTT_length += 5

		if len(out) == 0:
			## valid RTT not found
			self.no_RTT = True
			print ("No valid RTT found given current max length:%s"%(max_max_RTT_length))
			return 0
		self.RTT_df = pd.DataFrame(out)
		self.RTT_df.columns = ['chr','start','end','target_to_RTT5']
		self.RTT_df["strand"] = get_opposite_strand(self.strand)
		self.RTT_df.index = ["%s_RTT_%s"%(self.uid,i) for i in range(self.RTT_df.shape[0])]
		if debug>10:
			print (self.target_fa)
			print (self.user_target_pos)
		temp = get_fasta_simple(self.target_fa,self.RTT_df, self.user_target_pos)
		self.RTT_df['old_seq'] = temp[3].tolist()

		## add variant
		# relative_pos = self.target_pos-r['start']-1 # start is 0-index
		if debug>10:
			pd.set_option('display.max_columns', None)
			print (self.RTT_df)
		self.RTT_df['seq'] = [self.add_variant(r['old_seq'],self.target_pos-r['start']-1,self.ref,self.alt) for i,r in self.RTT_df.iterrows()]
		self.RTT_df = self.RTT_df[self.RTT_df['seq']!=0]
		if self.strand == "+": ## when sgRNA is positive strand, RTT should use the negative strand
			self.RTT_df['seq'] = [revcomp(x) for x in self.RTT_df['seq']]
		# print (self.RTT_df)
		self.RTT_df['RTT_length'] = [len(x) for x in self.RTT_df['seq'] ]
		if large_deletion_flag:
			self.RTT_df = self.RTT_df[self.RTT_df['RTT_length']<=user_max_RTT_length]
		self.RTT_df = self.RTT_df[self.RTT_df['RTT_length']>=min_RTT_length]
		# filter out first C
		self.RTT_df['firstC'] = [x[0] for x in self.RTT_df['seq']]
		self.RTT_df = self.RTT_df[self.RTT_df['firstC']!="C"]
		self.RTT_df = self.RTT_df.drop(['firstC'],axis=1)


		current_max_RTT_length = max_RTT_length+5
		while self.RTT_df.shape[0] == 0:
			if debug>=10:
				print ("increasing max_RTT_length to:", current_max_RTT_length)
			if current_max_RTT_length > max_max_RTT_length:
				break
			_,RTT_df,no_RTT_flag = self.find_longer_RTT(min_RTT_length=min_RTT_length,max_RTT_length=current_max_RTT_length,min_distance_RTT5=min_distance_RTT5,**kwargs)
			if RTT_df.shape[0] > 0:
				print ("max_RTT_length increased from %s to %s"%(max_RTT_length,current_max_RTT_length))
				self.RTT_df = RTT_df
				break
			current_max_RTT_length += 5



		if self.RTT_df.shape[0] == 0:
			## valid RTT not found
			self.no_RTT = True
			print ("NO RTT found: new RTT sequence length is longer than: %s"%(max_max_RTT_length))
			return 0


		## make features
		## modify it to fit fasta input
		attached_minimal_PBS = sub_fasta_single(self.target_fa,self.user_target_pos, pbs_start,pbs_end)
		if self.strand == "+": ## when sgRNA is positive strand, RTT should use the negative strand
			attached_minimal_PBS = revcomp(attached_minimal_PBS)
		# print ("attached_minimal_PBS",attached_minimal_PBS)
		# print (self.RTT_df)


		self.RTT_df['RNAfold_seq']= [(self.scaffold_seq+x+attached_minimal_PBS).replace("T","U") for x in self.RTT_df['seq']]
		RNAfold_features_df = pd.DataFrame([call_RNAplfold(x,len(self.scaffold_seq))for x in self.RTT_df['RNAfold_seq']])
		# print (RNAfold_features_df)
		RNAfold_features_df.index = self.RTT_df.index.tolist()
		self.RTT_df = pd.concat([self.RTT_df,RNAfold_features_df],axis=1)
		self.RTT_df['RTT_GC'] = [GC_content(x) for x in self.RTT_df['seq'] ]

		self.RTT_df.columns = [str(x) for x in self.RTT_df.columns]
		self.is_dPAM = is_dPAM(self.PAM, self.RTT_df['seq'][0], self.offset)
		# print (self.RTT_df)



#find RTT sequences as bed format df هدف تابع
		#Output:
    #1.RTT_df
		#2.RTT_feature_list

	def find_longer_RTT(self,min_RTT_length=10,max_RTT_length=20,min_distance_RTT5=5,**kwargs):
		out = []
		chr = self.chr
		pbs_start = None
		pbs_end = None
		user_max_RTT_length = max_RTT_length
		large_deletion_flag=False
		if len(self.ref)+min_distance_RTT5 > max_RTT_length: ## in case of large deletion
			large_deletion_flag = True
			max_RTT_length = max_RTT_length+len(self.ref)
		# target_to_RTT5_feature=[]
		if self.strand=="+":
			start = self.cut_position # remember out cut position, the actual nucleotide, we use -4
			pbs_end = start
			pbs_start = pbs_end - 14
			for l in range(min_RTT_length,max_RTT_length+1):
				end = start+l
				if start+1<=self.target_pos <=end-min_distance_RTT5:
					out.append([chr,start,end,end-self.target_pos])
		if self.strand=="-":
			end = self.cut_position - 1
			pbs_start = end
			pbs_end = pbs_start + 14
			for l in range(min_RTT_length,max_RTT_length+1):
				start = end-l
				if end>=self.target_pos >=start+1+min_distance_RTT5:
					out.append([chr,start,end,self.target_pos-start-1])
		if len(out) == 0:
			return out,self.RTT_df,True
		self.RTT_df = pd.DataFrame(out)
		self.RTT_df.columns = ['chr','start','end','target_to_RTT5']
		self.RTT_df["strand"] = get_opposite_strand(self.strand)
		self.RTT_df.index = ["%s_RTT_%s"%(self.uid,i) for i in range(self.RTT_df.shape[0])]
		temp = get_fasta_simple(self.target_fa,self.RTT_df, self.user_target_pos)
		self.RTT_df['old_seq'] = temp[3].tolist()

		## add variant
		# relative_pos = self.target_pos-r['start']-1 # start is 0-index
		self.RTT_df['seq'] = [self.add_variant(r['old_seq'],self.target_pos-r['start']-1,self.ref,self.alt) for i,r in self.RTT_df.iterrows()]
		self.RTT_df = self.RTT_df[self.RTT_df['seq']!=0]
		if self.strand == "+": ## when sgRNA is positive strand, RTT should use the negative strand
			self.RTT_df['seq'] = [revcomp(x) for x in self.RTT_df['seq']]
		# print (self.RTT_df)
		self.RTT_df['RTT_length'] = [len(x) for x in self.RTT_df['seq'] ]
		if large_deletion_flag:
			self.RTT_df = self.RTT_df[self.RTT_df['RTT_length']<=user_max_RTT_length]
		self.RTT_df = self.RTT_df[self.RTT_df['RTT_length']>=min_RTT_length]
		# filter out first C
		self.RTT_df['firstC'] = [x[0] for x in self.RTT_df['seq']]
		self.RTT_df = self.RTT_df[self.RTT_df['firstC']!="C"]
		self.RTT_df = self.RTT_df.drop(['firstC'],axis=1)
		if self.RTT_df.shape[0] == 0:
			return out,self.RTT_df,True
		return out,self.RTT_df,False

#-----------------------------------------------------------------------------
		#check if target mutation overlaps with ngRNA and update its sequence	only work for len(ref) = len(alt), namely substitutions
		#return sequence
		# check if overlaps

	def make_PE3b_ngRNA(self,myIndex,start,end,seq,strand):
		target_pos_list = list(range(self.target_pos,self.target_pos+len(self.ref)))
		seq_pos_list = list(range(start+1,end+1))
		overlaps = set(seq_pos_list).intersection(target_pos_list)

		if len(overlaps) == 0:
			return [myIndex,seq,0]
		# print (seq,"contain overlaps",overlaps)
		if strand == "-":
			new_seq = list(revcomp(seq))
		else:
			new_seq = list(seq)
		for i in overlaps:
			relative_ngRNA_pos = i - start - 1
			relative_target_pos = i - self.target_pos
			if relative_ngRNA_pos < 0 or relative_target_pos<0:
				print ("Error: make_PE3b_ngRNA",relative_target_pos,relative_ngRNA_pos,self.sgRNA_name,start,end,new_seq,strand,self.variant_id)
				return [myIndex,seq,0]
				relative_ngRNA_pos = 0
			ngRNA_ref = new_seq[relative_ngRNA_pos]
			target_ref = self.ref[relative_target_pos]
			target_alt = self.alt[relative_target_pos]
			if target_ref != ngRNA_ref:
				print ("Error: target_ref != ngRNA_ref","%s != %s"%(target_ref,ngRNA_ref),relative_target_pos,relative_ngRNA_pos,start,end,new_seq,strand,self.variant_id)
				return [myIndex,seq,0]
			new_seq[relative_ngRNA_pos] = target_alt
		if strand == "-":
			new_seq = revcomp("".join(new_seq))
			return [myIndex,new_seq,1]
		else:
			new_seq = "".join(new_seq)
			return [myIndex,new_seq,1]



#------------------------------------------------------------------------------
	#find all valid ngRNAs given the sgRNA name هدف تابع
	#Input: ngRNA_df , gRNA_feature_list

	def find_nick_gRNA(self,max_ngRNA_distance=100,max_max_ngRNA_distance=200,debug=0,**kwargs):
		self.ngRNA_df = self.opposite_strand_sgRNAs.copy()
		# print (self.ngRNA_df)
		if self.ngRNA_df.shape[0] == 0:
			self.no_ngRNA = True
			print ("no ngRNA for %s"%(self.sgRNA_name))
			return 0
		self.ngRNA_df.index = ["%s_ngRNA_%s"%(self.uid,i) for i in range(self.ngRNA_df.shape[0])]
		self.ngRNA_df.columns = ['chr','start','end','seq','sgRNA_name','strand']
		if self.strand == "-":
			self.ngRNA_df['sgRNA_distance_to_ngRNA'] = [-self.dist_dict[x][self.sgRNA_name] for x in self.ngRNA_df["sgRNA_name"]]
		if self.strand == "+":
			self.ngRNA_df['sgRNA_distance_to_ngRNA'] = [self.dist_dict[x][self.sgRNA_name] for x in self.ngRNA_df['sgRNA_name']]
		# print (max_ngRNA_distance,"max_ngRNA_distance")
		current_ngRNA_df = self.ngRNA_df[self.ngRNA_df['sgRNA_distance_to_ngRNA'].abs()<=max_ngRNA_distance]
		current_max_ngRNA_distance = max_ngRNA_distance + 20
		while current_ngRNA_df.shape[0] == 0:
			if current_max_ngRNA_distance > max_max_ngRNA_distance:
				self.ngRNA_df = current_ngRNA_df
				break
			current_ngRNA_df = self.ngRNA_df[self.ngRNA_df['sgRNA_distance_to_ngRNA'].abs()<=current_max_ngRNA_distance]
			if current_ngRNA_df.shape[0] > 0:
				self.ngRNA_df = current_ngRNA_df
				print ("max ngRNA distance increased from %s to %s:"%(max_ngRNA_distance,current_max_ngRNA_distance))
				break
			current_max_ngRNA_distance += 20
		self.ngRNA_df = current_ngRNA_df
		if self.ngRNA_df.shape[0] == 0:
			print ("no ngRNA for max distance: %s"%(max_ngRNA_distance))
			self.no_ngRNA = True
			return 0

		self.ngRNA_df['is_PE3b'] = 0

		if len(self.ref) == len(self.alt): # check PE3b
			pe3b = pd.DataFrame([self.make_PE3b_ngRNA(		i,
																														r['start'],
																														r['end'],
																														r['seq'],
																														r['strand']
																													) for i,r in self.ngRNA_df.iterrows()])
			# print ("-"*20,self.variant_id,"-"*20)
			# print (pe3b)
			pe3b = pe3b.set_index(0)
			self.ngRNA_df[['seq','is_PE3b']] = pe3b[[1,2]]
	  	# print (self.ngRNA_df)




#-------------------------------------------------------------------------------
		#Steps:
		#Assume all input is on the positive strand
    #ref alt is the corrected version
		#e.g., GC - > C becomes C to "" (empty)
		#relative_pos 0-index

	def add_variant(self,RTT_seq,relative_pos,ref,alt,**kwargs):

		## function check
		if len(ref)>0:
			get_ref = RTT_seq[relative_pos:relative_pos+len(ref)]
			# print (self.sgRNA_name,"relative_pos",relative_pos,"get_ref",get_ref)
			if get_ref != ref:
				if len(get_ref)<len(ref):
					## large deletion, given RTT is not long enough
					return 0
				print (self.variant_id,self.sgRNA_name,"references do not match",RTT_seq,relative_pos,ref,alt)
				return 0
		if relative_pos < 0:
			print (self.variant_id,self.sgRNA_name,"relative position < 0, result will not be correct!")
			return 0
			relative_pos = 0
		new_RTT = RTT_seq[:relative_pos]	+ self.alt + RTT_seq[relative_pos+len(self.ref):]
		return new_RTT




#-------------------------------------------------------------------------------
#get rawX and X formated dataframe, we can use the itertools to take all combinations of the index then we concat them as rows, -> rawX 	we concat them as columns -> X
#input: PBS, RTT, ngRNA sequences and features
#output: self.rawX, 		self.X

	def get_rawX_and_X(self,debug=0,**kwargs):


		rawX_columns = ["seq","chr","start","end","strand"]

		if self.no_RTT:
			return 0
		## not allow PE2 cases
		if self.no_ngRNA:
			print ("no_ngRNA found",self.sgRNA_name)
			return 0
			self.ngRNA_df=pd.DataFrame([np.nan]*(len(self.ngRNA_feature_list)+len(rawX_columns))).T
			self.ngRNA_df.columns = rawX_columns + self.ngRNA_feature_list
		if debug>=10:
			print (self.variant_id,"showing PBS_df, RTT_df, and ngRNA df")
			print (self.PBS_df.head())
			print (self.RTT_df.head())
			print (self.ngRNA_df.head())
		X_index = []
		PBS_selected_rows =[]
		RTT_selected_rows =[]
		ngRNA_selected_rows =[]
		all_list = [self.PBS_df.index.tolist(),self.RTT_df.index.tolist(),self.ngRNA_df.index.tolist()]
		temp = list(itertools.product(*all_list))
		count = 0
		for s in temp:
			count += 1
			current_index = "%s_%s_candidate_%s"%(self.variant_id,self.sgRNA_name,count)
			# current_index = "%s_%s"%(self.variant_id,count) # shorter name
			if count == 1  and debug>10:
				print (current_index)
			PBS_selected_rows.append(s[0])
			RTT_selected_rows.append(s[1])
			ngRNA_selected_rows.append(s[2])
			if self.ngRNA_df.at[s[2],'is_PE3b']==1:
				current_index+="_PE3b"
				# print (s[2],"PE3B")
			if self.is_dPAM:
				current_index+="_dPAM"
				# print (self.sgRNA_name,"dPAM")
			if self.no_ngRNA:
				current_index+="_PE2"
			X_index.append(current_index)


		# ------------------------------------------------------------  rawX  ------------------------------------------------------------
		rawX_PBS = self.PBS_df.loc[PBS_selected_rows][rawX_columns]
		rawX_RTT = self.RTT_df.loc[RTT_selected_rows][rawX_columns]
		rawX_ngRNA = self.ngRNA_df.loc[ngRNA_selected_rows][rawX_columns]
		rawX_sgRNA = pd.DataFrame([self.seq,self.chr,self.start,self.end,self.strand]).T
		rawX_sgRNA.columns = rawX_columns
		rawX_sgRNA = rawX_sgRNA.loc[[0]*len(ngRNA_selected_rows)][rawX_columns]
		rawX_PBS['sample_ID']  = X_index
		rawX_RTT['sample_ID']  = X_index
		rawX_ngRNA['sample_ID']  = X_index
		rawX_sgRNA['sample_ID']  = X_index
		rawX = pd.concat([rawX_PBS,rawX_RTT,rawX_ngRNA,rawX_sgRNA])
		rawX['CHROM'] = self.chr
		rawX['POS'] = self.user_target_pos
		rawX['REF'] = self.user_ref
		rawX['ALT'] = self.user_alt
		rawX['type'] = ['PBS']*rawX_PBS.shape[0]+['RTT']*rawX_RTT.shape[0]+['ngRNA']*rawX_ngRNA.shape[0]+['sgRNA']*rawX_ngRNA.shape[0]
		self.rawX = rawX[["sample_ID","CHROM","POS","REF","ALT","type","seq","chr","start","end","strand"]]
		self.rawX = self.rawX.sort_values("sample_ID")
		self.rawX.index = self.rawX['sample_ID'].tolist()



		# ------------------------------------------------------------  X  ------------------------------------------------------------

		X_PBS = self.PBS_df.loc[PBS_selected_rows][self.PBS_feature_list]
		X_PBS = X_PBS.reset_index(drop=True)
		X_RTT = self.RTT_df.loc[RTT_selected_rows][self.RTT_feature_list]
		X_RTT = X_RTT.reset_index(drop=True)
		X_ngRNA = self.ngRNA_df.loc[ngRNA_selected_rows][self.ngRNA_feature_list]
		X_ngRNA = X_ngRNA.reset_index(drop=True)
		self.X = pd.concat([X_PBS,X_RTT,X_ngRNA],axis=1)
		self.X.index = X_index
		self.X['is_dPAM'] = self.is_dPAM
		self.X['target_to_sgRNA'] = self.target_to_sgRNA
		self.X['DeepSpCas9'] = self.DeepSpCas9
		print ("kkkkkkkkkkkkkkkkk")
		print ("init",sgRNA_name)


# Target mutation

In [ ]:
#تعدادی از ویژگی های تاثیر گذار در دقت خروجی ویرایش مد نظر اینجا تعریف می شوند و تعدادی هم که در سلول بالا و کلاس sgRNA تعریف شده اند
# (nick_to_pegRNA', 'target_to_pegRNA', 'target_to_RTT5','aln_ref_alt_mis', 'aln_ref_alt_del', 'aln_ref_alt_ins',  'PBS_GC', 'RTS_GC','PBS_length', 'RTS_length', 0, 1, 2, 3, 4, 5, 6, 7,
# (nick_to_pegRNA,target_to_pegRNA,'is_dPAM' ,'aln_ref_alt_mis', 'aln_ref_alt_del', 'aln_ref_alt_ins')define here

# (target_to_RTT5, RTS_GC, RTS_length,0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13 ) defined  in sgRNA class / find_RTT
# (PBS_GC,PBS_length):  defined in sgRNA class /find_PBS



#solve the problem when user specification of ref, alt contain redundancy ATTTT-> ATTT, should be T -> "" G - > GC will  be "" C
def find_mutation_pos(pos,ref,alt):
	count=0
	for i in range(min(len(ref),len(alt))):
		x=ref[i]
		y=alt[i]
		if x != y:
			return pos,ref[i:],alt[i:]
		else:
			pos+=1
			count+=1
	return pos,ref[count:],alt[count:]


class target_mutation:
	def __init__(self,chr,pos,name,ref,alt,target_fa,**kwargs):
	  #sgRNA name: chr_start_end_strand_seq
		#target_mutation name: id_chr_pos_ref_alt
    #pos is corrected, and the corrected pos, ref, alt is used
    #	target_fa is the +-1000 extended sequences

		self.chr = chr
		self.target_pos = pos
		self.name = name.replace("/","_").replace(",","_")
		self.ref = ref
		self.alt = alt
		self.target_fa = target_fa
		self.debug_folder = "easy_prime_debug_files"
		self.dist_dict = {}
		self.strand_dict = {}
		self.rawX = pd.DataFrame()
		self.X = pd.DataFrame()
		self.X_p = pd.DataFrame()
		self.topX = pd.DataFrame()
		self.allX = pd.DataFrame()
		self.pegRNA_flag=True
		## flags
		self.found_PE3b = False
		self.found_PE3 = False
		self.found_PE2 = False
		self.found_dPAM = False
		self.N_sgRNA_found = 0


		# self.feature_for_prediction = ["sgRNA_distance_to_ngRNA","target_to_sgRNA","target_to_RTT5","N_subsitution","N_deletion","N_insertions","PBS_GC","RTT_GC","PBS_length","RTT_length",'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',"is_dPAM"] # match the order of training features
		self.feature_for_prediction = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','DeepSpCas9',"sgRNA_distance_to_ngRNA","is_dPAM",'is_PE3b','RTT_GC', 'RTT_length', 'PBS_GC', 'PBS_length', 'N_subsitution', 'N_deletion', 'N_insertions',"target_to_sgRNA","target_to_RTT5"] # match the order of training features
		# self.feature_rename = ["ngRNA_pos","Target_pos","Target_end_flank","N_subsitution","N_deletion","N_insertions","PBS_GC","RTT_GC","PBS_length","RTT_length",'Folding_DS_1', 'Folding_DS_2', 'Folding_DS_3', 'Folding_DS_4', 'Folding_DS_5', 'Folding_DS_6', 'Folding_DS_7', 'Folding_DS_8', 'Folding_DS_9','Folding_DS_10',"is_dPAM"]
		self.PE3_model_feature_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'cas9_score', 'nick_to_pegRNA', 'dPAM', 'PE3b', 'RTT_GC', 'RTT_length', 'PBS_GC', 'PBS_length', 'N_subsitution', 'N_deletion', 'N_insertions', 'Target_pos', 'Target_end_flank']
		self.PE2_model_feature_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'cas9_score', 'RTT_GC', 'RTT_length', 'PBS_GC', 'PBS_length', 'N_subsitution', 'N_deletion', 'N_insertions', 'Target_pos', 'Target_end_flank','dPAM']



		self.mutation_pos,self.mutation_ref,self.mutation_alt = find_mutation_pos(pos,ref,alt)
		self.sgRNA_strand_df={}
		self.sgRNA_strand_df["+"]=pd.DataFrame()
		self.sgRNA_strand_df["-"]=pd.DataFrame()
		self.valid_init_sgRNA = pd.DataFrame()
		self.all_sgRNA = pd.DataFrame() # used to find ngRNA

		#---------------- features --------------------------------------------------------------------------------------------------------------------------
		# target mutation feature
		self.ref_alt = global_alignments(self.ref,self.alt)
		self.sgRNA_target_distance_dict = {} ## contain valid and invalid sgRNA in the key, but the latter with  distance <0
		self.DeepSpCas9_dict = {}
		self.sgRNA_target_dPAM_dict = {} ## contain binary values of whether the target affect this sgRNA PAM
		# sgRNA distance to ngRNA
		self.dist_dict = {} ## sgRNA_ngRNA_distance_dict


	def init(self,gRNA_search_space=200,search_iteration=1,sgRNA_length=20,PAM="NGG",offset=-3,debug=0,genome_fasta=None,max_RTT_length=40,min_distance_RTT5=5,max_target_to_sgRNA=10,max_max_target_to_sgRNA=30,**kwargs):
		#first step: search sgRNA
		#second step: search PBS
		#search for candidate sgRNAs around target mutation
		#Input:1-	gRNA_search_space: extend pos by +- gRNA_search_space
    #2-search_iteration: if in the search space defined by gRNA_search_space, we fail to find sgRNAs, we will extend the gRNA_search_space further to find at least one sgRNA. (no need to increase it)
		#Output:chr, start, end, sgRNA name, seq, strand, cut_position, valid
		#These will be used later: self.offset ,self.PAM

		if debug>0:
			subprocess.call("mkdir -p %s"%(self.debug_folder),shell=True)
			self.offset = offset
		self.PAM = PAM


		### find all sgRNA given a sequence
		for i in range(search_iteration):
			extend = gRNA_search_space*(i+1)
			if i >=1:
				print ("No sgRNA were found using %s gRNA_search_space"%(extend))
			## modified for fasta input
			start = max(self.mutation_pos-extend,0)
			end = self.mutation_pos+extend
			if len(self.target_fa) <= extend*2:
				search_fa = self.target_fa
				start = 0
			else:
				search_fa = sub_fasta_single(self.target_fa,self.target_pos, start,end)
			df = run_pam_finder(search_fa,"N"*sgRNA_length,self.PAM,start,self.chr)
			## df contains all sgRNAs
			self.N_sgRNA_found = df.shape[0]

			if df.shape[0] > 0:
				self.DeepSpCas9_dict = get_DeepSpCas9_score(df[4].unique().tolist())
			try:
				df[1] = df[1].astype(int)
				df[2] = df[2].astype(int)
				## sgRNA name
				df[4] = df[0]+"_"+df[1].astype(str)+"_"+df[2].astype(str)+"_"+df[5].astype(str)+"_"+df[3].astype(str)
				df.index = df[4].to_list()
				df['cut'] = [get_gRNA_cut_site(x[1],x[2],x[5],self.offset) for i,x in df.iterrows()]
				df['target_distance'] = [is_gRNA_valid([r[0],r['cut']],[self.chr,self.mutation_pos],r[5],self.target_pos,len(self.mutation_ref)) for i,r in df.iterrows()]


				## gRNA validation given target mutation
				if debug > 5:
					print ("total sgRNA found (contain invalid sgRNAs): %s"%(df.shape[0]))
					df.to_csv("%s/%s.init.all_sgRNAs.bed"%(self.debug_folder,self.name),sep="\t",header=False,index=False)

				self.valid_init_sgRNA = df[df.target_distance.between(1,max_target_to_sgRNA)][[0,1,2,3,4,5,'cut']]
				current_max_target_to_sgRNA = max_target_to_sgRNA+5

				while self.valid_init_sgRNA.shape[0] == 0:
					if debug>=10:
						print ("increasing max_target_to_sgRNA to:", current_max_target_to_sgRNA)
					if current_max_target_to_sgRNA > max_max_target_to_sgRNA:
						break
					self.valid_init_sgRNA = df[df.target_distance.between(1,current_max_target_to_sgRNA)][[0,1,2,3,4,5,'cut']]
					if self.valid_init_sgRNA.shape[0] > 0:
						print ("max_target_to_sgRNA increased from %s to %s"%(max_target_to_sgRNA,current_max_target_to_sgRNA))
						break
					current_max_target_to_sgRNA += 5
				## sgRNA features
				self.sgRNA_target_distance_dict = df['target_distance'].to_dict()

				if debug > 5:
					print ("showing sgRNAs between 1 to %s"%(current_max_target_to_sgRNA))
					print (df[df.target_distance.between(1,current_max_target_to_sgRNA)])
				df = df.drop(['target_distance'],axis=1)
				if self.valid_init_sgRNA.shape[0] == 0:
					print ("No sgRNA was found for %s using %s gRNA_search_space"%(self.name,extend))
					continue
				else:
					self.found_PE2 = True
					print ("%s valid sgRNAs found for  %s"%(self.valid_init_sgRNA.shape[0],self.name))
					self.dist_dict = distance_matrix(df.values.tolist())
					self.sgRNA_strand_df['+'] = df[df[5]=="+"][[0,1,2,3,4,5]]
					self.sgRNA_strand_df['-'] = df[df[5]=="-"][[0,1,2,3,4,5]]
					self.all_sgRNA = df.copy()
					# self.sgRNA_target_dPAM_dict = {i: is_dPAM(PAM_seq, RTT, self.offset) for i, r in self.valid_init_sgRNA.iterrows()}
					# self.sgRNA_target_dPAM_dict = {i: is_dPAM(self.PAM, self.target_pos,self.ref,self.alt,r[0:4].tolist()) for i, r in self.valid_init_sgRNA.iterrows()}


					break

			except Exception as e:
				print (e)
				print ("Error or No sgRNA was found for %s using %s gRNA_search_space"%(self.name,extend))

		if debug > 5:
			print ("Target name: ",self.name)
			print (self.valid_init_sgRNA.head().to_string(index=False))


	def search(self,debug=0,scaffold=None,**kwargs):
		#Second step: search for all possible PBS, RTS, pegRNA, nick-gRNA combos
		#Input:length min and max to define search space
		#Output:1. valid sgRNA list
		#     	2. PBS dataframe
		#       3. RTT dataframe
		#       4. ngRNA dataframe

		if not self.found_PE2:
			return 0


		self.sgRNA_list = [sgRNA(
								chr = x[0],
								start = x[1],
								end = x[2],
								seq = x[3],
								sgRNA_name = x[4],
								strand = x[5],
								cut_position = x[6],
								mutation_pos = self.mutation_pos,mutation_ref = self.mutation_ref,mutation_alt = self.mutation_alt,
								user_target_pos = self.target_pos,user_ref = self.ref,user_alt = self.alt,
								offset = self.offset,target_to_sgRNA = self.sgRNA_target_distance_dict[x[4]],
								variant_id = self.name,
								dist_dict = self.dist_dict,
								opposite_strand_sgRNAs = self.sgRNA_strand_df[get_opposite_strand(x[5])],
								all_sgRNA_df = self.all_sgRNA,
								target_fa = self.target_fa,
								scaffold_seq = scaffold,
								PAM = self.PAM,
								DeepSpCas9 = self.DeepSpCas9_dict[x[3]]
								)
						for x in self.valid_init_sgRNA.values.tolist()]

		[run_sgRNA_search(s,**dict(kwargs,debug=debug)) for s in self.sgRNA_list]

		self.rawX = pd.concat([s.rawX for s in self.sgRNA_list])
		if debug>=10:
			print (self.name,"combined rawX:")
			print (self.rawX.head())
		if self.rawX.shape[0]==0:
			self.found_PE2=False
			return 0
		self.X = pd.concat([s.X for s in self.sgRNA_list])
		no_ngRNA = sum([s.no_ngRNA for s in self.sgRNA_list])
		if no_ngRNA == len(self.sgRNA_list):
			print ("%s only PE2 found"%(self.name))
		else:
			self.found_PE3 = True


		self.X['N_insertions'] = self.ref_alt[2]
		self.X['N_subsitution'] = self.ref_alt[1]
		self.X['N_deletion'] = self.ref_alt[3]


		self.found_PE3b = (self.X['is_PE3b']==1).any()
		self.found_dPAM = (self.X['is_dPAM']==1).any()




#بعد پیدا کردن همه  رشته ها حالا باید اسکور آنها را پیش بینی کنیم.
	def predict(self,debug=0,PE2_model=None,PE3_model=None,**kwargs):
		if not self.found_PE2:
			return 0

		#مدل های ذخیره شده را اینجا باز می کند
		with open(PE2_model, 'rb') as file:
			xgb_model_PE2 = pickle.load(file)
		with open(PE3_model, 'rb') as file:
			xgb_model_PE3 = pickle.load(file)

		self.X = self.X[self.feature_for_prediction]
		self.X.columns = self.PE3_model_feature_names

		# Split into PE2 and PE3 feature matrix
		X_PE2 = self.X[self.X.nick_to_pegRNA.isnull()]
		X_PE3 = self.X[~self.X.nick_to_pegRNA.isnull()]

#تابع پیش بینی را اینجا فراخوانی می کنیم
		pred_y_PE2 = xgb_model_PE2.predict(X_PE2[self.PE2_model_feature_names])
		pred_y_PE3 = xgb_model_PE3.predict(X_PE3)

		myPred = pd.DataFrame()
	#نتایج هر 3 مدل با هم ترکیب می شود.
	#pred_y_PE2.tolist() OR pred_y_PE3.tolist()
		myPred['predicted_efficiency'] = pred_y_PE2.tolist()+pred_y_PE3.tolist()#+DNABERTmodel.tolist()+myoldmodel()
		myPred.index = X_PE2.index.tolist()+X_PE3.index.tolist()
		self.X_p = pd.concat([self.X,myPred],axis=1)
		self.rawX['predicted_efficiency'] = myPred.loc[self.rawX.index]['predicted_efficiency']


#این دو مقدار نهایی حاصل از کد من هست که باید پرینت شود.
		self.X_p = self.X_p.sort_values("predicted_efficiency",ascending=False)
		self.rawX = self.rawX.sort_values("predicted_efficiency",ascending=False)


		# میتوانی این قسمت ها را حذف کنی
		# recommend dPAM when ever possible
		tmp = self.rawX.copy()
		if self.found_dPAM:
			tmp = tmp[tmp.index.str.contains('dPAM')]
		# recommend PE3b except when its predicted efficiency is 10% smaller than the highest ones
		tmp['rank'] = tmp.apply(lambda r:force_recommend_dPAM_PE3b(r,tmp.predicted_efficiency.max()),axis=1)
		tmp = tmp.sort_values(['rank','predicted_efficiency'],ascending=False)
		tmp = tmp.drop(['rank'],axis=1)
		self.topX = tmp.loc[tmp.index[0]]



def run_sgRNA_search(s,**kwargs):
	s.find_RTT(**kwargs)
	s.find_PBS(**kwargs)
	s.find_nick_gRNA(**kwargs)
	s.get_rawX_and_X(**kwargs)


# Final model

In [ ]:
def print_parameters(myDict):
	myGroup = {}
	myGroup['Prime Editing'] = ['genome_fasta','scaffold','n_jobs','debug','PE2_model','PE3_model','extend_length']
	myGroup['PBS searching'] = ['min_PBS_length','max_PBS_length']
	myGroup['RTT searching'] = ['min_RTT_length','max_RTT_length','min_distance_RTT5','max_max_RTT_length']
	myGroup['sgRNA searching'] = ['gRNA_search_space','sgRNA_length','offset','PAM','max_target_to_sgRNA','max_max_target_to_sgRNA']
	myGroup['ngRNA searching'] = ['max_ngRNA_distance']
	for k in myGroup:
		print_group(myDict,myGroup[k],k)


def print_group(myDict,myList,group_title):
	print ("-------- Parameter Group: %s --------"%(group_title))
	for l in myList:
		print ("%s: %s"%(l,myDict[l]))


#تعریف پارامترهای پیش فرض
def get_parameters(config):
	p_dir = os.path.dirname(os.path.realpath(__file__)) + "/"
	# return dict
	parameters = {}
	# default parameters
	pre_defined_list = {}
	#------------ Prime Editing related-----------
	#این ورودی را باید برای برنامه فراهم کنم
	pre_defined_list["genome_fasta"] = "/home/yli11/Data/Human/hg19/fasta/hg19.fa"
	pre_defined_list["n_jobs"] = -1
	pre_defined_list["scaffold"] = "GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC"
	pre_defined_list["debug"] = 0
	pre_defined_list["extend_length"] = 1000 # extracting +- 1000bp center at target pos from the genome, in 99.9% cases, you don't need to change this. If change to less than 500, will trigger fasta input mode, may cause error.
	#   مدل های از پیش آموزش دیده را اینجا معرفی می کنم.
	# می تونی DEEPPRIME بزاری
	pre_defined_list["PE2_model"] = p_dir+"../model/PE2_model_final.py"
	pre_defined_list["PE3_model"] = p_dir+"../model/PE3_model_final.py"

	#------------ PBS -----------
	pre_defined_list["min_PBS_length"] = 10
	pre_defined_list["max_PBS_length"] = 15

	#------------ RTT -----------
	pre_defined_list["min_RTT_length"] = 10
	pre_defined_list["max_RTT_length"] = 20 # if no candidate is found, this value will be increased by 5, max to max_max_RTT_length
	pre_defined_list["max_max_RTT_length"] = 50
	pre_defined_list["min_distance_RTT5"] = 5

	#------------ sgRNA -----------
	pre_defined_list["gRNA_search_space"] = 200
	pre_defined_list["sgRNA_length"] = 20
	pre_defined_list["offset"] = -3
	pre_defined_list["PAM"] = "NGG"
	pre_defined_list["max_target_to_sgRNA"] = 10 # if no candidate is found, this value will be increased by 5, max to max_max_target_to_sgRNA
	pre_defined_list["max_max_target_to_sgRNA"] = 30

	#------------ ngRNA ------------
	pre_defined_list["max_ngRNA_distance"] = 100 # if no candidate is found, this value will be increased by 20, max to max_max_ngRNA_distance
	pre_defined_list["max_max_ngRNA_distance"] = 200
	pre_defined_list["search_iteration"] = 1 # not affect anything

	try:
		with open(config, 'r') as f:
			manifest_data = yaml.load(f,Loader=yaml.FullLoader)
	except:
		print ("Config data is not provided or not parsed successfully, Default parameters were used.")

	for p in pre_defined_list:
		try:
			parameters[p] = manifest_data[p]
		except:
			parameters[p] = pre_defined_list[p]
	return parameters


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import sys
import argparse
import datetime
import getpass
import os

#Output: The output folder will contain:
#1. all pegRNA + ngRNA combination for the input vcf file
#2. top1 pegRNA + ngRNA combination for each variant
#3. visualization of the top1s [TODO]
#4. a summary file of each variant

# گرفتن ورودی ها به شکل فایل
def my_args():
	mainParser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter,description="pegRNA design")
	username = getpass.getuser()

	mainParser.add_argument('-f','--input_file',  help="vcf or fasta",required=True)
	mainParser.add_argument('-c','--config',  help="A YAML file specifying parameters",default=None)

	mainParser.add_argument('-o','--output',  help="output dir",default="easy_prime_%s_%s_result_dir"%(username,str(datetime.date.today())))

	#add parameters above
	args = mainParser.parse_args()
	return args


# دو گامی که در سکشن بالا تعریف کرده ام را اینجا فراخوانی می کنم.
def run_steps(t,**kwargs):
	t.init(**kwargs)
	t.search(**kwargs)
	t.predict(**kwargs)
	return [t.topX,t.rawX,t.X_p,t.found_PE3b,t.found_PE3,t.found_dPAM,t.found_PE2,t.N_sgRNA_found]





NameError: ignored

In [ ]:

def main():

	#پارامترها را به صورت فایل میگیرم
  #برای هر تارگت کلاس بالا را فراخوانی می کنم
	args = my_args()
	# ------------------------- get parameters ----------------------------------------------

	parameters = get_parameters(args.config)
	print_parameters(parameters)

	try:
		vcf = pd.read_csv(args.input_file,comment="#",sep="\t",header=None)
		vcf[1] = vcf[1].astype(int)
		vcf =vcf.drop_duplicates(2) # remove duplicated names
		vcf[3] = [x.upper() for x in vcf[3]]
		vcf[4] = [x.upper() for x in vcf[4]]
		vcf[5] = vcf2fasta(vcf,**parameters)
		vcf = vcf[list(range(6))]

	except:
		try:
			print ("Reading fasta file: %s"%(args.input_file))
			vcf = fasta2vcf(args.input_file)
			print (vcf)
		except:
			print ("Can't read %s as vcf or fasta. Please check input. Exit..."%(args.input_file))
			exit()

	variant_list = vcf[2].tolist()


## for each target, create target mutation class
	my_targets = [target_mutation(*r) for i,r in vcf.iterrows()]



#find best pegRNAs
# backend can affect this parallization, if so, user show use n_jobs=1
	if parameters['n_jobs'] == 1:
		df_list = [run_steps(t,**parameters) for t in my_targets]
	else:
		from joblib import Parallel, delayed
		df_list = Parallel(n_jobs=parameters['n_jobs'],verbose=10)(delayed(run_steps)(t,**parameters) for t in my_targets)


#--------------------------------------------ضروری نیست می توانی این ها را حذف کنی-------------------------------------------------------------------------
	# save output
	#یا این کد ها و یا کدهای نوشته شده قبلی، فقط چاپ سطری در بالا
	import subprocess
	subprocess.call("mkdir -p %s"%(args.output),shell=True)
	summary = pd.DataFrame([x[3:8] for x in df_list]).astype(int)
	summary.columns = ['found_PE3b','found_PE3','found_dPAM','found_PE2',"N_sgRNA_found"]
	summary.index = variant_list
	summary.to_csv("%s/summary.csv"%(args.output),index=True)

	df_top = pd.concat([x[0] for x in df_list])
	if df_top.shape[0]==0:
		print ("no pegRNA were found for the input file: %s"%(args.input_file))
		sys.exit()
	df_top = df_top.sort_values("predicted_efficiency",ascending=False)
	df_top.to_csv("%s/topX_pegRNAs.csv"%(args.output),index=False)

	df_all = pd.concat([x[1] for x in df_list])
	df_all = df_all.sort_values("predicted_efficiency",ascending=False)
	df_all.to_csv("%s/rawX_pegRNAs.csv.gz"%(args.output),index=False,compression="gzip")

	X_p = pd.concat([x[2] for x in df_list])
	X_p = X_p.sort_values("predicted_efficiency",ascending=False)
	X_p.to_csv("%s/X_p_pegRNAs.csv.gz"%(args.output),index=True,compression="gzip")




if __name__ == "__main__":
	main()

#visualization

In [ ]:
!pip install dna_features_viewer

In [ ]:
#visualization
import warnings
warnings.filterwarnings("ignore")
import sys
import argparse
import datetime
import getpass
import os
from dna_features_viewer import GraphicFeature, GraphicRecord
import pandas as pd
import uuid
import subprocess
import matplotlib.pyplot as plt
npg_colors = ["#E64B35","#4DBBD5","#00A087","#3C5488","#F39B7F","#464d4f"]
my_colors = {}
my_colors['sgRNA'] = npg_colors[0]
my_colors['PBS'] = npg_colors[1]
my_colors['RTT'] = npg_colors[2]
my_colors['ngRNA'] = npg_colors[3]
my_colors['variant'] = "#e6fc3f"
"""

Output
--------

The output folder will contain:
1. all pegRNA + ngRNA combination for the input vcf file
2. top1 pegRNA + ngRNA combination for each variant
3. visualization of the top1s [TODO]
4. a summary file of each variant

"""

def my_args():
	mainParser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter,description="easy_prime for pegRNA design")
	username = getpass.getuser()

	mainParser.add_argument('-f','--rawX',  help="input rawX format, last column with predicted efficiency",required=True)
	mainParser.add_argument('-s','--genome_fasta',  help="input genome sequences",required=True)
	mainParser.add_argument('-t','--figure_type',  help="png or pdf",default="png")
	mainParser.add_argument('--sample_id',  help="only plot this sample id",default=None)
	mainParser.add_argument('--output_file_name',  help="use only when plot one pegRNA, output_file_name.figure_type, will overwrite -t -o options.",default=None)
	mainParser.add_argument('-o','--output',  help="output dir",default="easy_prime_vis_%s_%s"%(username,str(datetime.date.today())))

	##------- add parameters above ---------------------
	args = mainParser.parse_args()
	return args

def write_file(file_name,message):
	out = open(file_name,"wt")
	out.write(message)
	out.close()
def get_fasta_single(chr,start,end,genome_fasta=None):
	out_bed = str(uuid.uuid4()).split("-")[-1]
	out_fa = str(uuid.uuid4()).split("-")[-1]
	write_file(out_bed,"%s\t%s\t%s"%(chr,start,end))
	command = "bedtools getfasta -fi %s -bed %s -fo %s -tab"%(genome_fasta,out_bed,out_fa)
	subprocess.call(command,shell=True)
	lines = open(out_fa).readlines()[0]
	seq = lines.split()[-1]
	subprocess.call("rm %s;rm %s"%(out_bed,out_fa),shell=True)
	return seq
def get_strand(x):
	if x == "+":
		return 1
	else:
		return -1
def find_mutation_pos(pos,ref,alt):
	"""solve the problem when user specification of ref, alt contain redundancy
	ATTTT-> ATTT, should be T -> ""
	G - > GC will  be "" C
	"""
	count=0
	for i in range(min(len(ref),len(alt))):
		x=ref[i]
		y=alt[i]
		if x != y:
			return pos,ref[i:],alt[i:]
		else:
			pos+=1
			count+=1
	return pos,ref[count:],alt[count:]

def plot_main(df,output=None,genome_fasta=None,figure_type="png",output_file_name=None,**kwargs):
	"""Given one instance of Prime Editing prediction (rawX format), generate DNA visualization

	Input
	--------
	the data frame contains 4 rows: RTT, PBS, sgRNA, ngRNA

	"""
	pegRNA_id = df.index.tolist()[0]
	variant_id = pegRNA_id.split("_")[0]
	chr = df['CHROM'][0]
	start = df['start'].min()
	start -= start%10
	start -= 1
	start = max(start,0)
	end = df['end'].max()
	end -= end%10
	end += 10
	df = df.fillna("")
	variant_pos = df.POS.min()
	ref = df.REF[0]
	alt = df.ALT[0]
	try:
		correct_pos,new_ref,new_alt = find_mutation_pos(variant_pos,ref,alt)
	except:
		correct_pos,new_ref,new_alt = variant_pos,ref,alt
	predicted_efficiency = df.predicted_efficiency[0]*100
	pos = variant_pos-start
	pos_rel_correct = correct_pos-start
	sequence = get_fasta_single(chr,start,end,genome_fasta).upper()

	feature_list = []
	for s,r in df.iterrows():
		r_start = r.start-start
		r_end = r_start+(r.end-r.start)
		r_strand = get_strand(r.strand)
		gf = GraphicFeature(start=r_start, end=r_end, strand=r_strand,
			color=my_colors[r.type],label=r.type)
		feature_list.append(gf)
	record = GraphicRecord(sequence=sequence, features=feature_list)

	ax, _ = record.plot(figure_width=int(len(sequence)/5))
	record.plot_sequence(ax)
	if len(new_ref) > 0:
		for xxx in range(len(new_ref)):
			ax.fill_between((pos_rel_correct-1.5+xxx, pos_rel_correct-0.5+xxx), +1000, -1000, alpha=0.5,color=my_colors['variant'])
	else:
		ax.fill_between((pos-1.5, pos-0.5), +1000, -1000, alpha=0.5,color=my_colors['variant'])
	locs, labels = plt.xticks()
	new_labels = []
	flag = True
	for i in locs:
		if flag:
			new_labels.append("%s %s"%(chr,int(start+i+1)))
			flag=False
		else:
			new_labels.append(int(start+i+1))
	plt.xticks(locs,new_labels)
	dPAM=""
	if "dPAM" in pegRNA_id:
		dPAM = "PAM disruption"
	if "PE3b" in pegRNA_id:
		dPAM += ", PE3b"
	if dPAM!="":
		dPAM += "\n"
	myTitle = "ID: %s, CHR: %s, POS: %s, REF: %s, ALT: %s \n %s Predicted efficiency: %.1f"%(variant_id,chr,variant_pos,ref,alt,dPAM,predicted_efficiency)+"%"


	plt.title(myTitle)
	if output_file_name!= None:
		ax.figure.savefig(output_file_name, bbox_inches='tight')
	else:
		ax.figure.savefig(f'{output}/{pegRNA_id}.{figure_type}', bbox_inches='tight')


def main():

	args = my_args()
	if not os.path.isfile(args.genome_fasta):
		print (f"genome fasta NOT FOUND: {args.genome_fasta}")
	df = pd.read_csv(args.rawX,index_col=0)
	subprocess.call(f"mkdir -p {args.output}",shell=True)
	for i in df.index.unique().tolist():
		if args.sample_id != None:
			if i != args.sample_id:
				continue
		print (f"Processing {i}...")
		plot_main(df.loc[i],**vars(args))


if __name__ == "__main__":
	main()

usage: ipykernel_launcher.py [-h] -f RAWX -s GENOME_FASTA [-t FIGURE_TYPE]
                             [--sample_id SAMPLE_ID]
                             [--output_file_name OUTPUT_FILE_NAME] [-o OUTPUT]
ipykernel_launcher.py: error: the following arguments are required: -s/--genome_fasta


SystemExit: ignored